# Goals

use self-validation of LLM to improve the accuracy of data extraction. Aiming to improve from 55% to 80%.

# Function: Trim to Relevant Part - Case Type Specific

In [0]:
import re


def trim_judgment(full_text):
    """
    Sequentially applies text trimming based on the presence of specific keywords or their absence.

    Search for "书记员" with the constraint that the characters can be spaced out but must all occur within 20 characters of the sequence.
    Search for "查明" and, if found, trim everything before this keyword in the already trimmed text (from the "书记员" search).
    If "查明" is not found, trim the first third of the text that might have been already trimmed based on the "书记员" search.

    Returns:
    - trimmed_text (str): The result of the trimming operation.
    - trimmedType (str): The method of trimming ('书记员', '查明', or 'last 2/3').
    - text_around_trimmed_point (str): Context around the point of trimming.
    """

    trim_a_third_switch = True

    if full_text is None or not isinstance(full_text, str):
        return None, "ERROR", ""

    # Initialize default values
    trimmed_text = full_text
    text_around_trimmed_point = ""
    trimmedType = "None"

    # Search for "书记员" with characters within 20 characters of each other
    matches = list(re.finditer(r"书\s{0,18}记\s{0,18}员", full_text))
    if matches:
        
        # Take the last match since we are interested in the occurrence starting from the end
        clerk_index = matches[-1].end()
        # Context around "书记员"
        start_pos = max(0, clerk_index - 10)
        end_pos = min(len(trimmed_text), clerk_index + 10)
        text_around_trimmed_point = trimmed_text[start_pos:end_pos]
        
        trimmed_text = trimmed_text[:clerk_index]  # Trim everything after the last "书记员"
        trimmedType = "书记员"


    # Search for "查明" in the already trimmed text
    find_index = trimmed_text.find("查明")

    if find_index > 0 and trimmed_text[find_index - 1] == "另":
    # "查明" is part of "另查明", adjust find_index or handle as needed
        pass

    elif find_index != -1:
        # Context around "查明"
        start_pos = max(0, find_index - 10)
        end_pos = min(len(trimmed_text), find_index + 10)
        text_around_trimmed_point = trimmed_text[start_pos:end_pos]
        
        trimmed_text = trimmed_text[find_index:]  # Trim before "查明"
        trimmedType = "查明"

    elif trimmedType != "查明":  # Only trim 1/3 if "查明" wasn't found
        if trim_a_third_switch:
            one_third_length = len(trimmed_text) // 3
            
            # Context around the trimmed point
            start_pos = max(0, one_third_length - 10)
            end_pos = min(len(trimmed_text), one_third_length + 10)
            text_around_trimmed_point = trimmed_text[start_pos:end_pos]
            
            # Trim the first 1/3
            trimmed_text = trimmed_text[one_third_length:]
            trimmedType = "last 2/3"


    return trimmed_text, trimmedType, text_around_trimmed_point

# Function: Post Request and fetch Claude Response

In [0]:
import time
import requests
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType

api_key = dbutils.secrets.get(scope = "OhMyGPTAPI", key = "OhMyGPTAPI1")

def trim_and_fetch_facts(judgment_text):
    
    trimmed_judgment_text, trimmedType, text_around_trimmed_point = trim_judgment(judgment_text)

    url = "https://cfcus02.opapi.win/v1/messages"
    payload = json.dumps({
        "model": "claude-3-haiku-20240307",
        "stream": False,
        "system": "user will give you a legal judgement in Chinese. Please extract the name and amount (in grams) of drugs from the judgment and output them and the evidence (qoute from source text) within a JSON object. Use JSON format with the keys drug_a, amount_a, and drug_b, amount_b, evidence. Be concise Do not add anything else to the reply. If you can't extract the kind of drug or amount. reply 'NA'. reply using Chinese.",
        "messages": [
            {"role": "user", "content": trimmed_judgment_text}, 
            {"role": "assistant", "content": "{"}
            ],
        "max_tokens": 4096
    })
    headers = {
        "Authorization": 'Bearer ' + api_key,
        'Content-Type': 'application/json'
    }

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=20)
            response.raise_for_status()
            parsed_response = json.loads(response.text)
            # Check if the expected keys/path exists
            if "content" in parsed_response and len(parsed_response["content"]) > 0 and "text" in parsed_response["content"][0]:
                response_text = parsed_response["content"][0]["text"]
            else:
                raise KeyError("Unexpected response structure")
            return response_text, trimmedType, text_around_trimmed_point
        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"Error on attempt {attempt+1}: {e}")
        time.sleep(2 ** attempt + 1)

    # Default values in case of failure
    return "Failed to fetch response from API", "Error", "Error"



# read csv and extract drug type and amount in bunk

In [0]:
import os
import pandas as pd

import json

def parse_response_text(response_text):
    # Strip leading and trailing whitespace/newlines before fixing the format
    corrected_format = "{" + response_text.strip().replace("'", '"')
    try:
        # Parse the corrected string as JSON
        data = json.loads(corrected_format)
    except json.JSONDecodeError:
        # Return None or a dictionary with default values if parsing fails
        return None
    return data


def process_files(base_path, cause):
    global total_rows_count, failed_fetch_count, na_in_response_count
    # Define the output folder based on the base path and cause
    output_folder = f"{base_path}/{cause}_DrugTypeAmount_March_19"
    
    # Path pattern to match CSV files for the cause of action
    path_pattern = os.path.join(base_path, cause)
    
    # List to collect DataFrames (if needed)
    dfs = []

    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(path_pattern)

    for file in files:
        # Check if the item is a directory and iterate through its contents
        if file.isDir():
            sub_files = dbutils.fs.ls(file.path)
            for sub_file in sub_files:
                if sub_file.name.endswith(".csv"):
                    # Reading CSV file into DataFrame
                    file_path = sub_file.path.replace("dbfs:", "/dbfs")  # Convert to local file path if necessary
                    df = pd.read_csv(file_path, on_bad_lines='skip')

                    
                    # Filter rows where 'TrialProcedure' column contains '一审'
                    df_filtered = df[df['TrialProcedure'].str.contains('一审', na=False)]

                    # Apply 'trim_and_fetch_facts' function to 'FullText' column
                    enriched_data = df_filtered['FullText'].apply(trim_and_fetch_facts)

                    if df_filtered.empty:
                        print(f"No data after filtering for {sub_file.name}. Moving to the next file.")
                        continue

                    if not isinstance(enriched_data.iloc[0], tuple) or len(enriched_data.iloc[0]) != 3:
                        print(f"Data structure mismatch in file: {file_path}")
                        continue
                    
                    # Expanding the enriched_data into separate columns
                    df_expanded = pd.DataFrame(enriched_data.tolist(), columns=['ResponseText', 'TrimmedType', 'TextAroundTrimmedPoint'], index=df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_expanded], axis=1)
                    
                    # Parse the 'ResponseText' and expand it into new columns
                    parsed_data = df_filtered['ResponseText'].apply(parse_response_text)
                    df_parsed = pd.json_normalize(parsed_data).set_index(df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_parsed], axis=1)
                    
                    total_rows_count += len(df_filtered)
                    failed_fetch_count += len(df_filtered[df_filtered['ResponseText'] == 'Failed to fetch response from API'])
                    na_in_response_count += len(df_filtered[df_filtered['ResponseText'].str.contains("NA", na=False)])

                    # Drop the 'ResponseText' column
                    #df_filtered.drop('ResponseText', axis=1, inplace=True)

                    # Construct the output path for the enriched CSV file
                    output_file_path = 'dbfs:'+ os.path.join(output_folder, f"{os.path.basename(sub_file.name)}")
                    
                    # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
                    df_filtered.to_csv(output_file_path.replace("dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis"
causes_of_action = ["drug_related"]

for cause in causes_of_action:
    total_rows_count, failed_fetch_count, na_in_response_count = 0, 0, 0
    process_files(base_path, cause)

    # Print the counts for process failures.
    print(f"For {cause}, Total rows processed: {total_rows_count}")
    print(f"For {cause}, Failed API fetch count: {failed_fetch_count}")
    print(f"For {cause}, NA in response count: {na_in_response_count}")



# Tests

## Test trim and extract function

In [0]:
#API test block:
text_0 = "浙江省嵊州市人民法院 刑 事 判 决 书 （2019）浙0683刑初659号 公诉机关浙江省嵊州市人民检察院。 被告人王超，男，1991年11月17日出生，汉族，高中文化，无业，住宁夏回族自治区中卫市沙坡头区。因吸毒于2018年6月6日被嵊州市公安局行政拘留十一日，因吸毒于2018年11月21日被宁波市公安局奉化分局行政拘留十五日，因吸毒于2019年8月19日被嵊州市公安局行政拘留八日、强制戒毒二年。因本案于2019年8月10日被嵊州市公安局刑事拘留，9月16日被逮捕。现羁押于嵊州市看守所。 嵊州市人民检察院以嵊检二部刑诉[2019]56号起诉书指控被告人王超犯贩卖毒品罪，于2019年11月22日向本院提起公诉，并适用认罪认罚从宽制度，建议对被告人王超判处有期徒刑六个月，并处罚金人民币二千元。本院于11月25日立案，依法适用简易程序，实行独任审判，公开开庭审理了本案。嵊州市人民检察院指派检察员陈园出庭支持公诉，被告人王超到庭参加诉讼。现已审理终结。 嵊州市人民检察院指控，2018年7月20日下午，姚某用妻子单某的微信与被告人王超联系，委托被告人王超购买毒品甲基苯丙胺（冰毒），并支付被告人王超人民币650元。被告人王超从他人处获得0.1克甲基苯丙胺,于21日下午放置在嵊州市三江城&ldquo;东方巨龙&rdquo;对面的旗杆下，并将放置位置告诉姚某，由姚某捡走吸食。 2019年8月9日，被告人王超自动到嵊州市公安局甘霖派出所投案，如实供述了自己的犯罪事实。 上述事实，被告人王超在开庭审理过程中亦无异议，并有证人姚某、单某、王某、尹某的证言，微信转账记录，辨认笔录及照片，归案经过，情况说明，户籍证明，嵊州市公安局行政处罚决定书、强制隔离戒毒决定书等证据证实，足以认定。 本院认为，被告人王超违反毒品管理法律法规，贩卖毒品，其行为已构成贩卖毒品罪，应处三年以下有期徒刑，并处罚金。鉴于其犯罪以后自动投案，如实供述自己的罪行，是自首，予以从轻处罚。公诉机关指控的罪名成立，提出的量刑意见基本适当。根据其犯罪的事实、犯罪的性质、情节、对于社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一、四款、第三百五十七条、第六十七条第一款、第六十四条之规定，判决如下： 一、被告人王超犯贩卖毒品罪，判处有期徒刑六个月，并处罚金人民币二千元（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自二0一九年八月十日起至二0二0年二月九日止；罚金限判决生效后十日内缴纳）。 二、追缴被告人王超犯罪所得人民币650元，上缴国库。 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向浙江省绍兴市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。 审判员　　单学忠 二〇一九年十二月三日 书记员　　马生燕 附：《中华人民共和国刑法》相关法条： 第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。 走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产： （一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的； （二）走私、贩卖、运输、制造毒品集团的首要分子； （三）武装掩护走私、贩卖、运输、制造毒品的； （四）以暴力抗拒检查、拘留、逮捕，情节严重的； （五）参与有组织的国际贩毒活动的。 走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。 走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。 单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。 利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。 对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。 第三百五十七条本法所称的毒品，是指鸦片、海洛因、甲基苯丙胺（冰毒）、吗啡、大麻、可卡因以及国家规定管制的其他能够使人形成瘾癖的麻醉药品和精神药品。 毒品的数量以查证属实的走私、贩卖、运输、制造、非法持有毒品的数量计算，不以纯度折算。 第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。 被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。 犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。 第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 更多数据：www.macrodatas.cn"

text_1 ="云南省石屏县人民法院刑 事 判 决 书（2019）云2525刑初147号公诉机关石屏县人民检察院。被告人马威茂（绰号“小马”），男，1983年8月3日生，回族，初中文化，云南省峨山县人，客车驾驶员，住云南省峨山县。因涉嫌犯贩卖毒品罪，于2019年6月1日被刑事拘留，同年7月5日被逮捕，现羁押于石屏县看守所。辩护人张荣华，云南旭晖律师事务所律师。石屏县人民检察院以石检一部刑诉[2019]60号起诉书指控被告人马威茂犯贩卖毒品罪，于2019年10月21日向本院提起公诉。本院立案受理后，依法组成合议庭，于2019年12月19日公开开庭进行了审理。石屏县人民检察院委派查林娇、李涵出庭执行职务，被告人马威茂及其辩护人张荣华到庭参加诉讼。本案现已审理终结。石屏县人民检察院指控：被告人马威茂违反国家对毒品管理的规定，向多人多次贩卖甲基苯丙胺共计11.94克，其行为触犯了《中华人民共和国刑法》第三百四十七条第一款、第三款、第七款的规定，应当以贩卖毒品罪追究其刑事责任。被告人马威茂被抓获后如实供述其犯罪事实，系坦白，可以从轻处罚。建议对被告人马威茂在有期徒刑九年至十一年之间量刑，并处罚金30000元至50000元。公诉机关就指控的事实提交了相应的证据进行质证。被告人马威茂对公诉机关的指控无异议，当庭自愿认罪，但辩解其感觉没有贩卖过这么多毒品，请求从轻判处。辩护人张荣华对公诉机关指控被告马威茂犯贩卖毒品罪无异议。但辩护认为：马威茂本人系吸毒人员，量刑时应考虑其吸食毒品的情节；鉴于本案系零星交易毒品的案件，由于人的记忆力的局限性，不排除重复计算的可能性，难以精准确认毒品的数量，在量刑时应按照有利于被告人的原则对其从轻处罚；被告人马威茂一贯表现良好、系初犯，具有坦白情节。建议对马威茂在有期徒刑三年至七年之间量刑，并处罚金。经审理查明：被告人马威茂本人系吸毒人员，从事客车驾驶员工作。2018年至2019年5月30日期间，被告人马威茂向多人多次贩卖甲基苯丙胺片剂，具体事实如下：一、贩卖给李某（一）2019年4、5月份的一天，被告人马威茂在石屏县文化广场其客车上，以160元的价格贩卖甲基苯丙胺片剂4粒（计0.36克）给李某。（二）2019年4、5月份的一天，被告人马威茂在石屏县文化广场其客车上，以200元的价格贩卖甲基苯丙胺片剂6粒（计0.54克）给李某。（三）2019年4、5月份的一天，被告人马威茂在石屏县上，以200元的价格贩卖甲基苯丙胺片剂6粒（计0.54克）给李某。（四）2019年4、5月份，被告人马威茂在石屏县文化广场其客车上，先后2次以100元3粒的价格贩卖甲基苯丙胺片剂共计6粒（计0.54克）给李某。（五）2019年5月28日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。（六）2019年5月29日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。（七）2019年5月30日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给李某。二、贩卖给管某（一）2019年4月份，被告人马威茂先后3次以100元3粒的价格贩卖甲基苯丙胺片剂共计9粒（计0.81克）给管某。（二）2019年5月下旬的一天，被告人马威茂在石屏县文化广场其客车上，以450元的价格贩卖甲基苯丙胺片剂15粒（计1.35克）给管某。（三）2019年5月27日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给管某。（四）2019年5月29日14时许，被告人马威茂在石屏县文化广场其客车上，以300元的价格贩卖甲基苯丙胺片剂10粒（计0.9克）给管某。三、贩卖给董某1（一）2019年5月19日13时30分许，被告人马威茂在石屏县文化广场其客车上，以120元的价格贩卖甲基苯丙胺片剂4粒（计0.36克）给董某1。（二）2019年5月22日13时许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某1。（三）2019年5月22日13时30分许，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某1。（四）2019年5月22日23时许，被告人马威茂在峨山县，以600元的价格贩卖甲基苯丙胺片剂18粒（计1.62克）给董某1。四、贩卖给董某2（一）2019年5月28日14时许，被告人马威茂在石屏县文化广场其客车上，以200元的价格贩卖甲基苯丙胺片剂5粒（计0.45克）给董某2。（二）2019年5月29日14时许，被告人马威茂在石屏县文化广场其客车上，以50元的价格贩卖甲基苯丙胺片剂2粒（计0.18克）给董某2。五、贩卖给解某12018年以来，被告人马威茂先后2次贩卖甲基苯丙胺片剂共计6粒（计0.54克）给解某1。六、贩卖给解某22019年5月下旬的一天，被告人马威茂在石屏县文化广场其客车上，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给解某2。七、贩卖给董某32019年5月29日14时许，被告人马威茂在石屏县仁和旅舍，以100元的价格贩卖甲基苯丙胺片剂3粒（计0.27克）给董某3。2019年5月31日15时45分，被告人马威茂驾驶云Ｆ×××××号中型客车在石屏县龙某公路1KM+100M处被民警查获，当场从其身上查获一包用透明塑料自封袋封装的毒品可疑物17粒、白色OPPO牌手机一部、人民币1000元。经称量和鉴定，查获的17粒甲基苯丙胺片剂可疑物净重1.59克，含甲基苯丙胺成分。经侦查实验，被告人马威茂贩卖的甲基苯丙胺片剂每粒净重0.09克。上述款物被石屏县公安局扣押，并随案移送白色OPPO牌手机一部、人民币1000元至本院。另查明：一、经尿液检测，被告人马威茂的尿液检测结果冰毒、吗啡均呈阳性。二、被告人马威茂在贩卖毒品的过程中，使用VIVO牌手机与买方联系。三、被告人马威茂到案后如实供述自己的犯罪事实，当庭自愿认罪。上述事实，经当庭举证、质证，有下列证据：一、石屏县公安局出具的抓获经过、现场检查笔录、物证照片、扣押决定书、扣押清单、受案登记表、立案决定书；石屏县人民检察院随案移送物品清单。证实：2019年5月31日15时45分，石屏县公安局民警在石屏县龙某公路1KM+100M处查获马威茂，当场从其右裤包内查获一包用透明塑料自封袋封装的毒品可疑物17粒、左后裤包内查获一张农行卡、右后裤包查获人民币1000元，从其驾驶的云Ｆ×××××号客车驾驶室旁垫子上查获一部白色OPPO牌手机。石屏县公安局扣押了查获的上述款物，并于同日立为刑事案件侦查。石屏县人民检察院随案移送白色OPPO牌手机一部、人民币1000元至本院。二、证人证言（一）证人李某的证言，主要内容为：2019年5月20日左右，他向“小马”买过五、六次“小麻”，有一次在石屏县文化站门口“小马”的班车上以160元（微信转账）买了4粒“小麻”；有一次在石屏县文化站门口“小马”的班车上以200元（150元微信转账、50元现金）买了6粒“小麻”；有一次在石屏县旧沙桥头以200元（微信转账）买了6粒“小麻”；其他的都是以100元3粒的价格，在石屏县文化站门口“小马”的班车上买。2019年5月28日13时30分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”。2019年5月29日13时40分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元价买了3粒“小麻”。2019年5月30日14时50分许，他到石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”。（二）证人管某的证言，主要内容为：他吸食的“小麻”是在石屏县文化站门口的班车上向“小马”买的。2019年4月20日左右，他到石屏县文化广场门口向“小马”买了3粒“小麻”；另外两次也是2019年4月份，每次都是3粒，都是100元买的，都是他到班车上坐在“小马”后边，他递钱给“小马”，“小马”递东西给他。2019年5月份下旬向“小马”买了七、八次，都是微信红包或转账支付，最多的一次转账450元，购买了15粒，其余的300元、200元、100元不等，基本上都是以100元购买3粒的价格购买。2019年5月27日13时40分许，他到石屏县文化广场门口“小马”的班车上向“小马”以100元买了3粒“小麻”。2019年5月29日14时许，他到石屏县文化广场门口“小马”的班车上向“小马”以300元买了10粒“小麻”。（三）证人董某1的证言，主要内容为：2019年5月19日13时30分许，他在石屏县文化站门口“小马”的班车上向“小马”以120元（40元现金、80元微信转账）买了4粒“小麻”。2019年5月22日13时30分许，他在石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”；当天，他又在石屏县文化站门口“小马”的班车上向“小马”以100元买了3粒“小麻”；同日23时50分，他在峨山广场雕像处向“小马”以600元（微信转账）买了18粒“小麻”。（四）证人董某2的证言，主要内容为：2019年5月28日14时许，他在石屏县文化站门口“小马”的班车上向“小马”以200元买了5粒“小麻”。2019年5月29日14时许，他在石屏县文化站门口“小马”的班车上向“小马”以50元买了2粒“小麻”。自从他认识“小马”就开始向“小马”购买“小麻”，认识三年了，至少买过100粒左右“小麻”。（五）证人解某1的证言，主要内容为：两年前，他在龙武去峨山的岔路口“小马”的班车上向“小马”以160元买了4粒“小麻”；之后买了十多次，最少三两个，最多十个，价格有时35元1个，有时100元3个。（六）证人解某2的证言，主要内容为：2019年5月27日13时30分，他到石屏县文化站门口“小马”的班车上向“小马”以100元（微信转账）买了3粒“小麻”。2019年5月28日14时许，他到石屏县文化站门口“小马”的班车上向“小马”以100元（微信转账）买了3粒“小麻”。（七）证人董某3的证言，主要内容为：2019年5月29日14时许，他在石屏县仁和旅舍大门头上那个房间向“小马”以100元买了3粒“小麻”。三、被告人马威茂的供述和辩解，主要内容为：（一）他多次卖过“小麻”给李某。2019年4月份，有一次在石屏县文化广场门口其班车上以160元卖了4粒“小麻”给李某，有一次在石屏县文化广场门口其班车上以200元（50元现金、150元微信支付）卖了6粒“小麻”给李某，有一次在龙武旧沙××公路上以200元（微信支付）卖了6粒“小麻”给李某，后来在石屏县文化广场门口其班车上还卖过两次给李某，以每次100元3粒的价格卖了6粒。2019年5月28日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。2019年5月29日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。2019年5月30日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给李某。（二）2019年4月份，他卖过三次“小麻”给管某，以每次100元3粒的价格卖了9粒“小麻”给管某。2019年5月下旬，在石屏县文化广场门口其班车上，以450元（微信支付）卖了15粒“小麻”给管某。2019年5月29日14时许，在石屏县文化广场门口其班车上以300元卖了10粒“小麻”给管某。2019年5月27日13时许，在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给管某。（三）2019年5月19日13时30分许，他在石屏县文化广场门口其班车上以120元（40元现金、80元微信支付）卖了4粒“小麻”给董某1。2019年5月22日13时许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给董某1。2019年5月22日13时30分许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”。2019年5月22日23时许，他在峨山广场雕像处以600元（微信支付）卖了18粒“小麻”给董某1。（四）2019年5月28日14时许，他在石屏县文化广场门口其班车上以200元卖了5粒“小麻”给董某2。2019年5月29日14时许，他在石屏县文化广场门口其班车上以50元卖了2粒“小麻”给董某2。（五）一年前，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某1。前不久，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某1。（六）2019年5月28日13时许，他在石屏县文化广场门口其班车上以100元卖了3粒“小麻”给解某2。（七）2019年5月29日14时许，他在龙武某和旅舍以100元卖了3粒“小麻”给董某3。（八）他以前卖的“小麻”和被抓获时他装着的“小麻”是一模一样。四、辨认笔录及辨认照片，证实：管某、李某、解某1、解某2、董某3、董某1、董某2分别辨认出马威茂是卖“小麻”给其的人。马威茂分别辨认出李某、解某1、董某3、董某1、解某2是向其买“小麻”的人。五、毒品可疑物提取笔录、称量笔录及称量照片、侦查实验笔录及照片、鉴定委托书、毒品检验鉴定意见书、鉴定意见通知书等书证，证实：经称量，从马威茂处查获的毒品可疑物片剂17粒，净重1.59克；随机提取其中的1粒进行称量，净重0.09克。随机提取从马威茂处查获的毒品可疑物片剂作为检材送检，从检材中均检出甲基苯丙胺成分，鉴定意见已通知马威茂。六、尿液检验笔录及现场检测报告书，证实：2019年5月31日，民警依法对马威茂当场排出的尿液进行检测，检测结果冰毒、吗啡均呈阳性。七、户口证明、无违法犯罪记录证明，证实：马威茂的出生时间等基本自然情况，其作案时已达完全刑事责任年龄，在辖区居住期间无违法犯罪记录。以上证据来源合法，购买毒品人员的证言与被告人马威茂的供述中关于毒品交易时间、交易次数、交易数量存在部分不吻合，本院综合全案证据并根据证据之间相互能够印证、有利于被告人的原则作出认定，其它证据内容客观、真实，与案件事实相关联，具有证明力，本院予以确认。本院认为，被告人马威茂无视国家法律，违反国家对毒品管理的规定，向多人多次贩卖甲基苯丙胺共计11.94克，根据《中华人民共和国刑法》第三百五十七条“本法所称的毒品，是指鸦片、海洛因、甲基苯丙胺（冰毒）、吗啡、大麻、可卡因以及国家规定管制的其他能够使人形成瘾癖的麻醉药品和精神药品”、第三百四十七条第一款“走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚”、第三款“走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年有期徒刑以上有期徒刑，并处罚金”、第七款“对于多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算”的规定，被告人马威茂的行为已构成贩卖毒品罪，应据此规定予以相应的刑事处罚。公诉机关的指控事实清楚，证据确实充分，指控罪名成立。被告人马威茂被抓获后如实供述自己的罪行，根据《中华人民共和国刑法》第六十七条第三款的规定，系坦白，可以对其从轻处罚；其向多人多次贩卖毒品，可酌情从重处罚；其本人系吸毒人员，对从其处查获的毒品应当计入贩卖数量，但量刑时应考虑其吸食毒品的情节。在对被告人马威茂具体量刑时，本院根据马威茂贩卖毒品的事实、情节、社会危害程度、认罪悔罪态度等，予以综合考虑。公诉机关对量刑情节的认定准确、量刑建议适当，本院予以采纳。被告人马威茂“关于感觉没有贩卖过这么多毒品”的辩解意见，本院不予采信；其请求从轻判处的意见，本院予以综合考虑。辩护人张荣华关于“马威茂本人系吸毒人员，量刑时应考虑其吸食毒品的情节”以及“马威茂具有坦白情节”的辩护意见，本院予以采纳；其它辩护意见及量刑建议，与本院查明的事实及法律规定不符，本院不予采纳。公安机关扣押的毒品、毒资及犯罪工具，根据《中华人民共和国刑法》第六十四条的规定，应当予以没收。综上所述，本院为维护社会管理秩序，打击毒品犯罪，根据《中华人民共和国刑法》第三百五十七条、第三百四十七条第一款、第三款、第七款、第六十七条第三款、第六十四条之规定，判决如下：一、被告人马威茂犯贩卖毒品罪，判处有期徒刑九年，并处罚金人民币50000元。(刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2019年6月1日起至2028年5月31日止。罚金限判决生效之日一次性缴纳。)二、石屏县公安局扣押保管的17粒甲基苯丙胺片剂（净重1.59克），以及随案移送至本院的白色OPPO牌手机一部、人民币1000元，予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向云南省红河哈尼族彝族自治州中级人民法院提出上诉。书面上诉的，应交上诉状正本一份，副本二份。审　判　长　　王永才人民陪审员　　赵　昆人民陪审员　　杨官银二〇一九年十二月二十五日书　记　员　　白　龙 关注公众号“马 克 数 据 网”"

text_2 = "四川省崇州市人民法院 刑 事 判 决 书 （2019）川0184刑初625号 公诉机关四川省崇州市人民检察院。 被告人王磊，男，1984年3月6日出生，汉族，初中文化，住四川省崇州市。2019年4月26日因涉嫌犯非法持有毒品、容留他人吸毒罪被崇州市公安局刑事拘留，同年5月31日被逮捕。现羁押于崇州市看守所。 辩护人龙华，四川法邦律师事务所律师。 崇州市人民检察院以崇检公诉刑诉〔2019〕614号起诉书指控被告人王磊犯非法持有毒品罪、容留他人吸毒罪，于2019年12月23日向本院提起公诉。本院于同日立案，依法组成合议庭，适用简易程序，公开开庭审理了本案。崇州市人民检察院指派检察员杨萌出庭支持公诉，被告人王磊及其辩护人龙华到庭参加诉讼。现已审理终结。 崇州市人民检察院起诉指控：2019年4月19日左右，被告人王磊在崇州市容留吸毒违法人员赵某吸食毒品甲基苯丙胺（冰毒）；2019年4月22日左右，被告人王磊在唐兴客馆815房间容留吸毒人员赵某吸食冰毒；2019年4月25日，被告人王磊在崇州市容留吸毒违法人员岳某岚、塔某初吸食冰毒。 2019年4月25日11时许，崇州市公安局民警在对被告人王磊所租住的崇州市学苑东路229号1栋**房、**号出租房进行检查时，现场挡获吸毒人员王磊、岳某岚、塔某初，现场查获疑似毒品冰毒7袋、麻古3袋、吸毒工具一套。经称重，所查获的疑似毒品冰毒净重237.51克、麻古净重4.68克。后经成都市公安局物证鉴定所鉴定，上述毒品及吸毒工具内均含有甲基苯丙胺成分。 公诉机关认为，被告人王磊的行为应当以非法持有毒品罪、容留他人吸毒罪追究其刑事责任，建议对被告人王磊判处有期徒刑八年八个月至九年二个月，并处罚金三万四千元至五万四千元。公诉机关提交了公安机关的受案登记表和关于被告人王磊到案经过的说明，证人岳某岚、塔某初、赵某、倪某的证言，辨认笔录，鉴定意见，现场检查笔录和照片，扣押及保管清单，公安机关情况说明，被告人王磊的供述等证据证实，足以认定。 被告人王磊对指控的事实、罪名及量刑建议没有异议，同意适用简易程序且签字具结，在开庭审理过程中也无异议，自愿认罪认罚，但辩称自己曾检举孙某制造毒品的犯罪事实，应构成立功。辩护人龙华对指控的事实、罪名及量刑建议亦无异议，同意被告人王磊的辩解意见。 经审理查明的事实、证据与公诉机关的指控一致。另查明，公安机关出具了情况说明，证实公安机关已于2018年7月掌握孙某的犯罪线索并展开侦查，而被告人王磊是在2019年4月26日被羁押之后反映孙伟的犯罪线索。另，公安机关扣押在案被告人王磊随身携带的现金94,000元，在向银行缴纳时，被检查出其中一张百元面值为假币，已由银行没收，现扣押在案93,900元，暂存于崇州市公安局银行帐户；扣押手机两部（蓝色&ldquo;BIXING&rdquo;及黑色&ldquo;苹果&rdquo;手机各一部），以上物品均为被告人王磊个人财物；另扣押毒品甲基苯丙胺237.51克、麻古4.68克。扣押的手机与毒品均暂存于崇州市公安局涉案财物保管中心。 本院认为，被告人王磊违反国家毒品管理制度，非法持有含甲基苯丙胺成份的毒品，其行为构成非法持有毒品罪；同时被告人王磊为吸毒人员吸食毒品提供场所，其行为违反国家毒品管理制度，构成容留他人吸毒罪。崇州市检察院指控被告人王磊犯非法持有毒品罪和容留他人吸毒罪的事实和罪名成立，本院予以支持。关于被告人王磊提出自己检举孙某犯罪事实的行为是立功的辩解意见，本院认为，公安机关在被告人王磊检举孙某犯罪之前已经掌握了孙某的的犯罪线索，不属于立功，因此对被告人王磊的辩解意见不予采纳。辩护人龙华关于被告人王磊系初犯，归案后如实供述犯罪事实，自愿认罪认罚，请求对其从轻处罚的辩护意见与审理查明的事实一致，符合法律规定，予以采纳。被告人王磊一人犯数罪，予以数罪并罚。公诉机关量刑建议适当，予以采纳。根据被告人王磊犯罪的事实、性质、情节以及对于社会的危害程度，依照《中华人民共和国刑法》第三百四十八条、第三百五十四条、第六十九条、第六十七条第三款、第五十二条、第五十三条、第六十四条之规定，判决如下： 一、被告人王磊犯非法持有毒品罪，判处有期徒刑八年四个月，并处罚金三万六千元；犯容留他人吸毒罪，判处有期徒刑八个月，并处罚金四千元；数罪并罚，决定执行有期徒刑八年十个月，并处罚金四万元。 〔刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自二0一九年四月二十六日（羁押之日）起至二0二八年二月二十五日止。罚金限于本判决生效之次日起三日内缴纳。〕 二、对公安机关扣押在案的毒品甲基苯丙胺237.51克、麻古4.68克，由公安机关予以没收销毁；对公安机关扣押在案的手机两部（蓝色&ldquo;BIXING&rdquo;及黑色&ldquo;苹果&rdquo;手机各一部）发还被告人王磊，扣押在案的现金93,900元，用以执行罚金，剩余部分发还被告人王磊。 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接上诉于四川省成都市中级人民法院。书面上诉的，应当提交上诉状正本一份，副本二份。 审　判　长　　卫翔宇 人民陪审员　　吴　磊 人民陪审员　　钟　静 二〇一九年十二月二十四日 书　记　员　　龚　玲 附：《中华人民共和国刑法》 第三百四十八条非法持有鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的，处七年以上有期徒刑或者无期徒刑，并处罚金；非法持有鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。 第三百五十四条容留他人吸食、注射毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金。 第六十七条第三款犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。 第五十二条判处罚金，应当根据犯罪情节决定罚金数额。 第五十三条罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。 由于遭遇不能抗拒的灾祸等原因缴纳确实有困难的，经人民法院裁定，可以延期缴纳、酌情减少或者免除。 第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 第六十九条判决宣告以前一人犯数罪的，除判处死刑和无期徒刑的以外，应当在总和刑期以下、数刑中最高刑期以上，酌情决定执行的刑期，但是管制最高不能超过三年，拘役最高不能超过一年，有期徒刑总和刑期不满三十五年的，最高不能超过二十年，总和刑期在三十五年以上的，最高不能超过二十五年。 数罪中有判处附加刑的，附加刑仍须执行，其中附加刑种类相同的，合并执行，种类不同的，分别执行。 百度搜索“马 克 数 据 网”"

text_3 = "浙江省宁波市鄞州区人民法院 刑 事 判 决 书 （2019）浙0212刑初1388号 公诉机关浙江省宁波市鄞州区人民检察院。 被告人蒋磊，男，1983年9月19日出生于浙江省宁波市，汉族，初中文化，户籍所在地浙江省宁波市奉化区，住浙江省宁波市海曙区。2014年12月31日因吸毒被宁波市公安局海曙分局行政拘留五日。2015年3月19日因吸毒被宁波市公安局海曙分局行政拘留十五日。2015年6月17日因犯贩卖毒品罪被宁波市海曙区人民法院判处有期徒刑三年，并处罚金人民币一万元，2017年9月7日刑满释放。因涉嫌犯贩卖毒品罪于2019年3月20日被宁波市公安局鄞州分局传唤，次日被刑事拘留，同年4月26日被逮捕。现羁押于宁波市看守所。 辩护人王寅华，浙江甬港律师事务所律师，由宁波市鄞州区法律援助中心指派。 宁波市鄞州区人民检察院以甬鄞检刑诉〔2019〕1011号起诉书指控被告人蒋磊犯贩卖毒品罪，于2019年12月13日向本院提起公诉。本院于2019年12月17日立案，依法组成合议庭，适用普通程序，公开开庭审理了本案。庭审中被告人蒋磊表示自愿认罪，本案转为简易程序审理。宁波市鄞州区人民检察院指派检察员刘芙蓉出庭支持公诉，被告人蒋磊及其辩护人王寅华到庭参加诉讼。现已审理终结。 公诉机关指控： 2019年1月20日21时许，被告人蒋磊在宁波市海曙区荣安佳境小区门口，以人民币2000元的价格将2个甲基苯丙胺（冰毒）贩卖给陈某。 2019年3月4日18时许，被告人蒋磊在宁波市海曙区荣安佳境小区门口，以人民币1000元的价格将1个甲基苯丙胺贩卖给陈某。 同月20日，被告人蒋磊在宁波市海曙区被民警抓获。 上述事实，被告人蒋磊在开庭审理过程中亦无异议，且有证人陈某的证言，辨认笔录，技侦资料，刑事判决书，抓获经过，被告人蒋磊的身份证明等证据证实，足以认定。 本院认为，被告人蒋磊违反国家毒品管理法规，明知是毒品仍向他人非法贩卖，其行为已构成贩卖毒品罪。公诉机关指控的罪名成立。被告人蒋磊因贩卖毒品罪被判处有期徒刑，刑罚执行完毕后，五年内故意再犯应当判处有期徒刑以上的贩卖毒品罪，系累犯，又系毒品再犯，依法应当从重处罚。被告人蒋磊在庭审中能自愿认罪，可酌情从轻处罚。辩护人就此提出的辩护意见，本院予以采纳。根据被告人犯罪的事实、性质、情节和对于社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一款、笫四款、第三百五十六条、第六十五条第一款、第六十一条之规定，判决如下： 被告人蒋磊犯贩卖毒品罪，判处有期徒刑一年四个月，并处罚金人民币六千元。 （刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日。即自2019年3月20日起至2020年7月19日止。罚金限判决生效后一个月内向本院缴纳。） 如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向浙江省宁波市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。 审　判　长　　俞露烟 人民陪审员　　王锡莉 人民陪审员　　吴丽君 二〇一九年十二月三十日 代书　记员　　郑　莹 关注公众号“马 克 数 据 网”"

text_4 = "贵州省仁怀市人民法院刑 事 判 决 书（2017）黔0382刑初357号公诉机关贵州省仁怀市人民检察院。被告人杨应贵,女，1943年4月17日出生于贵州省仁怀市，汉族，文盲，无业，住贵州省仁怀市。2017年4月15日被仁怀市公安局刑事拘留，同年5月23日因涉嫌犯贩卖毒品罪被执行逮捕。现羁押于遵义市第二看守所。仁怀市人民检察院以仁怀检公刑诉〔2017〕292号起诉书指控被告人杨应贵犯贩卖毒品罪，于2017年9月7日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。仁怀市人民检察院指派检察员刘元良、助理检察员余静出庭支持公诉，被告人杨应贵到庭参加了诉讼。现已审理终结。经审理查明，被告人杨应贵系吸毒人员。2017年4月14日15时许，被告人杨应贵在其位于仁怀市茅坝镇茅坝社区东风街住宅三楼厨房内，以200元的价格贩卖毒品海洛因0.01克给罗小虎后被公安机关查获。公安机关在被告人处查获毒品海洛因1.11克、电子秤一台、现金1800元和直板手机一部。上述事实，有受案登记表、立案决定书、户籍证明、扣押决定书及清单、抓获经过、情况说明、现场检测报告书、被告人的供述和辩解、证人证言、辨认笔录、检查笔录、毒品称量笔录、鉴定文书、毒品收据等证据在卷佐证，且被告人在开庭审理过程中亦无异议，足以认定。本院认为，被告人杨应贵违反国家对毒品的管制制度，明知是毒品海洛因而予以贩卖，其行为已触犯《中华人民共和国刑法》第三百四十七条的规定，构成贩卖毒品罪。公诉机关指控的事实和罪名成立，本院予以确认。被告人归案后能如实供述自己的罪行，庭审中认罪态度较好，有悔罪表现，可对其从轻处罚。公诉机关建议对被告人判处七个月以上一年六个月以下有期徒刑并处罚金的公诉意见与本院查明的量刑情节一致，本院予以采纳。据此，本院为保护公民的身心健康，维护正常的社会管理秩序，依照《中华人民共和国刑法》第三百四十七条第四款、第六十七条第三款、第六十四条以及《最高人民法院关于适用财产刑若干问题的规定》第二条第一款的规定，判决如下：一、被告人杨应贵犯贩卖毒品罪，判处有期徒刑七个月，并处罚金二千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日。即自2017年4月15日起至2017年11月14日止。）二、涉案工具手机一部、电子秤一台，予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向贵州省遵义市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审　判　长　　刘　科人民陪审员　　杨元龙人民陪审员　　程天亮二〇一七年九月二十一日书　记　员　　赵小方 更多数据：www.macrodatas.cn"

text_5 = "湖南省冷水江市人民法院刑 事 判 决 书（2016）湘1381刑初192号公诉机关湖南省冷水江市人民检察院。被告人周丽，绰号“丽姐”，女，1981年4月8日出生，汉族，大专文化，无业，户籍所在地冷水江市,现住冷水江市；曾因犯贩卖毒品罪，2015年3月12日，被本院判处有期徒刑七个月，并处罚金五千元，2015年7月22日释放；现因涉嫌犯贩卖毒品罪，2016年9月12日被冷水江市公安局刑事拘留，同年9月27日被执行逮捕；现押于娄底市看守所。冷水江市人民检察院以冷检公诉刑诉［2016］194号起诉书指控被告人周丽犯贩卖毒品罪，于2017年11月7日向本院提起公诉。本院受理后，依法组成合议庭在本院审判庭公开开庭进行了审理。书记员林亚辉担任庭审记录，冷水江市人民检察院指派检察员刘丹丹出庭支持公诉，被告人周丽到庭参加诉讼。本案现已审理终结。冷水江市人民检察院指控：1、2016年9月9日16时许,吸毒人员刘某1通过微信联系被告人周丽求购毒品。后刘某1发了200元微信红包给周丽，周丽驾车将毒品甲基苯丙胺和毒品甲基苯丙胺片剂用卫生纸包好，要“三毛”将毒品藏匿于冷水江市中心小学附近，并通知刘某1去取。后刘某1赶至中心小学，在学校对面的垃圾桶旁取到毒品。2、2016年9月11日17时许，周丽电话联系吸毒人员周某，问周某是否需要毒品，周某遂求购300元毒品，双方约定在冷水江市富贵家园见面。见面后，周某发了300元微信红包给周丽，周丽随即叫来一辆出租车，在车上周丽将一粒毒品甲基苯丙胺片剂和两小包毒品甲基苯丙胺给了周某。交易完成后，周某回到了家中。2016年9月11日22时许，公安根据群众举报，在冷水江市轧钢加油站附近的莫林风尚酒店对面抓获被告人周丽，从周丽裤子袋子里缴获1小包白色晶体可疑物和小部分红色片状可疑物。经鉴定，白色晶体可疑物、小部分红色片状可疑物检出甲基苯丙胺成分。2016年9月12日，民警到周某家，扣押了周丽于9月11日贩卖给周某的0.89克白色晶体可疑物和0.09克红色片状可疑物。经鉴定，白色晶体可疑物、小部分红色片状可疑物检出甲基苯丙胺成分。为证明上述事实，公诉机关提交了刑事判决书、释放证明、微信交易截图、通话清单、户籍资料、毒品收缴收据等书证，证人周某、刘某1的证言，被告人周丽的供述与辩解，提取笔录、检查笔录、称重取样笔录、辨认笔录，抓获经过，情况说明等证据。公诉机关认为，被告人周丽贩卖毒品甲基苯丙胺，其行为触犯了《中华人民共和国刑法》第三百四十七条第一款、第四款之规定，应当以贩卖毒品罪追究其刑事责任。被告人周丽曾因犯贩卖毒品罪被判处有期徒刑以上刑罚，刑罚执行完毕以后，在五年内再犯应该判处有期徒刑以上刑罚之罪，根据《中华人民共和国刑法》第六十五条第一款、第三百五十六条、第七十四条的规定，系累犯、毒品再犯，应当从重处罚，且不适用缓刑。到案后，周丽如实供述其罪行，依法可以从轻处罚。提请本院依法惩处。被告人周丽对起诉书指控的犯罪事实供认不讳。经审理查明：1、2016年9月9日16时许，吸毒人员刘某1通过微信向被告人周丽求购毒品，并以微信转账的方式向周丽支付了200元毒资。随后，周丽驾车在新化县桑梓镇附近地点从一名绰号为“二哥”的男子处购得了重约0.8克的毒品甲基苯丙胺（冰毒）及一粒毒品甲基苯丙胺片剂（俗称“麻古”）。周丽从上述购得的毒品中克扣了约三分之一的毒品甲基苯丙胺后，将剩余毒品交给了一名绰号叫“三毛”的男子。随后，“三毛”在周丽的要求下，将上述毒品藏匿在了冷水江市中心小学大门对面一垃圾桶内。后刘某1按照周丽的通知，取走了藏匿在该处的毒品。2、2016年9月11日17时许，周丽电话联系吸毒人员周某，询问其是否需要购买毒品，周某表示同意，并与周丽约定在冷水江市富贵家园附近地段交易。两人来到约定地点后，周某以微信转账的方式向周丽支付了300元毒资。随即，周丽带领周某搭乘一辆出租车，在出租车上，周丽将两小包毒品甲基苯丙胺和一粒毒品甲基苯丙胺片剂交给了周某。后周某将上述购得毒品带回至其居住的房屋，并未吸食。2016年9月11日22时许，冷水江市公安局民警在冷水江市轧钢加油站旁的莫林风尚酒店对面附近地段将被告人周丽抓获。民警当场从周丽身上缴获并扣押了1小包毒品甲基苯丙胺疑似物及2小包毒品甲基苯丙胺疑似物与毒品甲基苯丙胺片剂疑似物的混合物。次日，民警在冷水江市计生委对面周丰居住的房屋内缴获并扣押了前日周丽贩卖给周某的2小包毒品甲基苯丙胺疑似物及1粒毒品甲基苯丙胺片剂疑似物。经称重，从周丽处扣押的毒品甲基苯丙胺疑似物净重0.09克，扣押的毒品甲基苯丙胺疑似物与毒品甲基苯丙胺片剂疑似物的混合物净重1.26克；从周某处扣押的毒品甲基苯丙胺疑似物净重0.89克，扣押的毒品甲基苯丙胺片剂疑似物净重0.09克。经鉴定，上述毒品疑似物均检测出甲基苯丙胺成分。综上所述，被告人周丽共贩卖毒品2次，贩卖毒品甲基苯丙胺重约2.95克。上述事实，有由公诉机关提交并经庭审质证、认证的下列证据予以证明：1、户籍资料，证明被告人周丽的基本情况。2、刑事判决书、释放证明，证明周丽曾因犯贩卖毒品罪，2015年3月12日，被本院判处有期徒刑七个月，并处罚金五千元，2015年7月22日释放。3、抓获经过，证明被告人周丽系被抓获归案。4、微信截图，证明周某通过微信向被告人周丽求购毒品以及支付毒资的情况。5、通话清单，证明被告人周丽与吸毒人员周某、刘某3案发时间段内有电话联系。6、毒品收缴收据、检查笔录、提取笔录、称重取样笔录，证明民警从被告人周丽及吸毒人员周某处缴获并扣押了毒品甲基苯丙胺疑似物及毒品甲基苯丙胺片剂疑似物若干。7、毒品鉴定意见，证明民警从被告人周丽及吸毒人员周某处扣押的毒品疑似物检测出甲基苯丙胺成分。8、辨认笔录，证明周某、刘某1均辨认出了向其贩卖毒品的被告人周丽；周丽辨认出了向其购买毒品的周某、刘某1。9、证人刘某1的证言，证明2016年9月9日下午，他通过微信向周丽求购毒品并支付了200元毒资。当天晚上，他接到周丽的电话后，按照周丽的指示在冷水江市中心小学门口对面的一个垃圾桶内找到了藏着的毒品“冰毒”和“麻古”，“冰毒”重约0.6至0.7克，“麻古”一粒。10、证人周某的证言，证明2016年9月11日下午，周丽通过电话联系问他是否需要购买毒品，他表示同意。后来他按照周丽的要求来到富贵家园附近。他和周丽见面后，他通过微信向周丽支付了300元毒资。然后，周丽叫了一辆出租车。他和周丽上车后，周丽将两小包“冰毒”和一粒“麻古”交给了他。他拿到毒品后没有吸食，而是带回了家中。10、被告人周丽的供述亦证明上述事实，与其他证据相互印证。上述证据，符合证据合法性、客观性与关联性的要求，本院予以确认。本院认为,被告人周丽贩卖毒品甲基苯丙胺，其行为已构成贩卖毒品罪，应依法予以惩处。被告人周丽曾因犯贩卖毒品罪被判处有期徒刑，现又犯贩卖毒品罪，系毒品犯罪再犯，依法应当从重处罚。被告人周丽在有期徒刑刑罚执行完毕以后，五年以内再犯应当判处有期徒刑以上刑罚之罪，且前后两罪均系故意犯罪，犯前罪时已满十八周岁，系累犯，依法应当从重处罚。被告人周丽到案后如实供述其罪行，依法可以从轻处罚。综上所述，公诉机关指控的罪名成立，应当以贩卖毒品罪追究被告人周丽的刑事责任。本院依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第七款，第三百五十六条，第六十五条第一款，第六十七条第三款，第五十二条，第五十三条，《中华人民共和国刑事诉讼法》第一百九十五条第（一）项之规定，判决如下：被告人周丽犯贩卖毒品罪，判处有期徒刑一年十个月，并处罚金人民币八千元。（刑期自判决执行之日起计算。判决执行前先行羁押的，羁押一日折抵刑期一日，即自2016年9月12日起至2018年7月11日止；罚金限判决生效之日起三十日内缴纳。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖南省娄底市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本两份。审　判　长　　肖若谷人民陪审员　　梁荷清人民陪审员　　谢林文二〇一七年九月二十五日书　记　员　　林亚辉附件：本判决所适用的相关法律法规：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第三百五十六条因走私、贩卖、运输、制造、贩卖毒品罪被判过刑，又犯本节规定之罪的，从重处罚。第六十五条被判处有期徒刑以上刑罚的犯罪分子，刑罚执行完毕或者赦免以后，在五年以内再犯应当判处有期徒刑以上刑罚之罪的，是累犯，应当从重处罚，但是过失犯罪和不满十八周岁的人犯罪的除外。前款规定的期限，对于被假释的犯罪分子，从假释期满之日起计算。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第五十二条判处罚金，应当根据犯罪情节决定罚金数额。第五十三条罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。由于遭遇不能抗拒的灾祸等原因缴纳确实有困难的，经人民法院裁定，可以延期缴纳、酌情减少或者免除。《中华人民共和国刑事诉讼法》第一百九十五条在被告人最后陈述后，审判长宣布休庭，合议庭进行评议，根据已经查明的事实、证据和有关的法律规定，分别作出以下判决：（一）案件事实清楚，证据确实、充分，依据法律认定被告人有罪的，应当作出有罪判决；（二）依据法律认定被告人无罪的，应当作出无罪判决；（三）证据不足，不能认定被告人有罪的，应当作出证据不足、指控的犯罪不能成立的无罪判决。 更多数据：www.macrodatas.cn"

text_6 = "上海市虹口区人民法院刑 事 判 决 书（2017）沪0109刑初778号公诉机关上海市虹口区人民检察院。被告人朱某某，男，1983年11月30日出生，汉族，户籍所在地安徽省淮南市。上海市虹口区人民检察院以沪虹检诉刑诉〔2017〕710号起诉书指控被告人朱某某犯贩卖毒品罪，于2017年8月31日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。上海市虹口区人民检察院指派检察员周某出庭支���公诉，被告人朱某某到庭参加诉讼。现已审理终结。上海市虹口区人民检察院指控：被告人朱某某受“小天”(另案处理)指使，于2017年6月20日14时30分许，在本市虹口区西江湾路XXX号“虹口龙之梦”商场门口，将2小包白色晶体以人民币800元的价格贩卖给俞某，双方成交后被民警当场抓获。案发后，经上海市公安局虹口分局称重，上述2包白色晶体净重1.56克；经上海市毒品检验中心鉴定，上述2包白色晶体中均检出甲基苯丙胺成分。上述事实，被告人朱某某在开庭审理过程中无异议，并有证人俞某、辛某的证言，上海市公安局虹口分局出具的《扣押笔录》、《扣押清单》、《称量笔录》、《案发经过》、《工作情况》，拍摄的相关照片、视频及上海市毒品检验中心出具的《检验报告》等证据证实，足以认定。本院认��，被告人朱某某明知是毒品而予以出售，其行为已构成贩卖毒品罪。上海市虹口区人民检察院指控被告人朱某某犯贩卖毒品罪罪名成立。本案系共同犯罪。被告人朱某某到案后能如实供述自己的罪行，依法可从轻处罚。为维护国家对毒品的管制，保护公民身心健康不受侵犯，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第二十五条第一款、第六十七条第三款及第六十四条之规定，判决如下：一、被告人朱某某犯贩卖毒品罪，判处有期徒刑六个月，并处罚金人民币二千元。(刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年7月1日起至2017年12月30日止。罚金应自本判决生效之日起一个月内向本院缴纳。)二、缴获的毒品及犯罪工具一并予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向上海市第二中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判员　　马翠华二〇一七年九月十一日书记员　　倪帼英附：相关法律条文《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。……走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。……第二十五条共同犯罪是指二人以上共同故意犯罪。……第六十七条……犯罪嫌疑人��不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。 来源：百度“马 克 数据网”"

text_7 = "湖北省红安县人民法院刑 事 判 决 书（2017）鄂1122刑初123号公诉机关湖北省红安县人民检察院。被告人柯桂菊，女，1969年1月4日出生，汉族，初中文化，无职业。出生地湖北省红安县，户籍所在地湖北省红安县，住湖北省红安县。因犯贩卖毒品罪，于2013年1月9日被武汉市江岸区人民法院判处有期徒刑六个月，并处罚金2000元，2013年3月14日刑满释放；因贩卖毒品罪，于2013年9月26日被武汉市江岸区人民法院判处有期徒刑八个月，并处罚金人民币4000元，2014年1月14日刑满释放。2017年3月30日因涉嫌贩卖毒品罪被刑事拘留，2017年5月5日被执行逮捕。现押黄冈市看守所。湖北省红安县人民检察院以鄂红检刑诉[2017]97号起诉书指控被告人柯桂菊犯贩卖毒品罪，于2017年8月2日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。湖北省红安县人民检察院指派检察员夏利和出庭支持公诉，被告人柯桂菊到庭参加了诉讼。本案现已审理终结。公诉机关指控：2016年4月至2017年3月期间，被告人柯桂菊在位于红安县八里湾镇阳福社区8号楼1单元4-1室的住处先后向红安县居民何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某等人销售麻果和冰毒等毒品牟利。2017年3月29日，红安县公安局干警现场抓获柯桂菊，并从其随身携带的黑色提包中查获59颗（共重5.51克）和一袋冰毒毒品（重0.27克）。随后，红安县公安局干警在被告人柯桂菊位于红安县红安县八里湾镇阳福社区8号楼1单元4-1室的住处搜查查获一颗麻果毒品（重0.11克）。经鉴定：从被告人柯桂菊随身携带的黑色提包中和住处查获、扣押的麻果和冰毒均检验出甲基苯丙胺毒品成分。公诉机关认为，被告人柯桂菊的行为构成贩卖毒品罪。被告人柯桂菊辩称，对公诉机关指控其犯贩卖毒品罪没有异议，但只向两人贩卖过毒品，一个是赵某，一个不知道叫什么名字，外号“黑子”或“黑皮”。不认识何某、韩某、刘某、徐某、胡某、袁某、张某、周某等人，也没有卖过毒品给他们。经审理查明：2016年4月至2017年3月期间，被告人柯桂菊在位于红安县八里湾镇阳福社区8号楼1单元4-1室的住处先后向红安县居民何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某等人销售麻果和冰毒等毒品牟利。2017年3月29日，红安县公安局干警现场抓获柯桂菊，并从其随身携带的黑色提包中查获59颗（共重5.51克）和一袋冰毒毒品（重0.27克）。随后，红安县公安局干警在被告人柯桂菊位于红安县红安县八里湾镇阳福社区8号楼1单元4-1室的住处搜查查获一颗麻果毒品（重0.11克）。经鉴定：从被告人柯桂菊随身携带的黑色提包中和住处查获、扣押的麻果和冰毒均检验出甲基苯丙胺毒品成分。上述事实，有经过法庭举证、质证的以下证据证实：1、证人何某的证言：我们都叫她嫂子或二嫂，红安口音，大约四十多岁，齐肩短发，一般身材。手机号码182××××9796。我在嫂子手中买过毒品。一次是2017年二月底，别人告诉问我的，把我带到红安县经济开发区二号路往八里方向去的一个小区里面，说四楼住着一个嫂子手中有毒品卖，然后我一个人上去，敲门，里面有个女的，问我做什么，我说买东西（毒品），她就问我要多少，我说两百元的，然后将200元钱给她，过了一会儿，她就从靠楼梯间的厕所窗户递出一包东西给我。里面装着四颗麻果。第二次是今年三月初的时候，我自己开车去的，以同样的方式买了一百元的毒品麻果两颗，并要到了嫂子的电话号码。第三次我先跟嫂子发信息，问她是否在家。嫂子回短信在家，我就开车过去，以上述相同的方式买了150元的毒品，麻果三颗。2017年3月24、5号的时候，韩某跟我打电话，要我帮忙带点毒品回去。我就直接去嫂子家，说要三百元钱的毒品，并将三百元给嫂子，她递给我四颗麻果和一袋子“冰”。我回到上新集，给了韩某三颗麻果和一袋子冰，自己留了一颗，韩某的三百元钱现在还没有给我。2、证人韩某的证言：3月25、6号左右，我叫何某跟我带了四颗麻果和一袋冰，他自己留了一颗，听她说是在二嫂手中买的。我认识二嫂，是个女的，住在觅儿工业园的一个小区里面，40多岁。我在二嫂手中拿过毒品。第一次是2016年4月份的时候，是陈佳（音）带我上去的，陈佳敲门，我跟陈佳将钱从窗户递进去，二嫂从窗户将毒品递出来。最近的一次是3月十几号的时候，我和何某一起去的，何某上楼拿货，我在下面等着。3、证人赵某的证言：我在八里工业园区阳福社区开了一家湖南米粉的早餐店。我在二嫂的手中购买过毒品，二嫂叫什么我不清楚，有40多岁，身高160左右，中等身材，卷发，染成黄色，长短刚到肩膀，红安本地口音。我一般就直接去二嫂居住的阳福社区四楼她家里去敲门或者打电话，她每次都是通过他家大门旁边的小窗户跟我沟通，我告诉她钱的金额后，她就将毒品给我，我就将现金通过窗户给她。4、证人刘某的证言：我与二嫂是在武汉做事的时候认识的，有二十年了，她大概四十多岁，真名字叫什么我不清楚，是红安人，住在工业园区阳福社区一栋居民楼的四楼。我在二嫂手中买过毒品。第一次是2017年春节前后，一天下午，我到二嫂居住的楼下，给她打电话说买点货，她问我要白的还是红的，我说都要，然后她从楼上给我送下来，一颗麻果一袋冰，我给了她100元。第二次是在第一次过后十几天，也是下午，我给二嫂打电话要买点麻果和冰，然后来到她家楼下，她从四楼将包好的毒品丢下来，我将100元钱放在她家楼下草坪的草下面就走了。第三次是2017年3月15日左右，我到了二嫂楼下，给她打电话说买多斤，她问我在什么地方，说她在外面买东西，然后在她们社区第一排房子旁边的过道上，卖给我一颗麻果和一袋冰，我给了她一百元钱。5、证人徐某的证言：二嫂大名叫可柯桂菊，是红安人，大概四十多岁，住在阳福社区第二排房子的四楼，电话是182××××9796，认识的人都叫她二嫂。我和她比较熟，是以前在武汉的时候认识的。我在柯桂菊手中买过两次毒品，第一次是2016年年底的时候，有一天下午我一个朋友从广州回来，他吸食毒品，我就给柯桂菊打电话说买点货。然后我来到她住的四楼，隔着窗户，她递给我一个塑料带，里面包着两颗麻果，我递给她100元现金。第二次是2017年春节后，我给柯桂菊打电话说要买点货，然后来到她住的四楼，还是跟上次一样，她隔着窗户递出一个包着毒品的塑料袋，里面包着两颗麻果，我从窗户递给她100元钱。6、证人胡某的证言：二嫂真名叫什么我不知道，是个四十多岁的女性，住在工业园区阳福社区8栋一单元401室，我有她的微信，在她被抓以后，我就将她的微信删了。我在二嫂手中买过毒品。第一次是2016年4月份的一天晚上，我和“猫子”开车来到二嫂楼下，我给了“猫子”300元钱上楼去买的一袋冰和四颗麻果。7、证人袁某（猫子）的证言：柯桂菊，女，一般大家都叫她二嫂，大概五十岁左右，住红安县工业园区阳福社区一栋楼的四楼。我跟她认识一年多了，是一个朋友“眼镜”带去买毒品时认识的，那是2016年的一天下午，我想要毒品，我听说眼镜在二嫂那里可以买到货，就给眼镜打电话，叫他带我去买。眼镜骑摩托车带我到阳福社区二嫂家楼下，上到四楼，眼镜敲门，隔着门跟二嫂说要货，买了两颗麻果、一袋冰，钱是我从窗户递进去的，二嫂将毒品包好从窗户递出来。8、证人张某的证言：二嫂大名好像叫柯桂菊，四十多岁，住工业园区阳福社区一栋楼的四楼，电话182××××9796，微信号叫“女王攻”。杜某是柯桂菊的儿子，大概二十多岁，跟二嫂住一栋楼，五楼。我在杜某手中买过三、四次毒品，他的毒品是从他母亲二嫂那里拿的。2017年3月份以前，买毒品的人都找柯桂菊，三月份以后，杜某开始卖毒品，但是不是帮他妈卖我就不清楚了。我是2015年经“猫子”介绍到柯桂菊手上买毒品。第一次是2015年，当时柯桂菊还住在觅儿岔路的旅馆那里，一天晚上，猫子带我到柯桂菊的住处，我在外面站着，给了180元给猫子，猫子进去找柯桂菊，过了一会儿，柯桂菊将三颗麻果包好送出来。9、证人周某的证言：我是通过“猫子”介绍认识二嫂的，二嫂大概四十多岁，女性，住工业园5号路旁边一个小区的四楼，到了那里我认得出来，她的电话和微信我都删除了。我在二嫂手中买多两次毒品，一次是2016年12月份的时候，一天下午，我先给二嫂打电话，说要四颗红的（指麻果），然后通过微信给二嫂转了200元钱，我来到二嫂居住的楼下，她从四楼阳台将用纸包好的麻果从楼上丢下来。第二次是在第一次之后2、3天，跟上次一样，我先打电话说要两颗红的，然后微信转100元给她，她从四楼将用纸包好的毒品丢下来。10、辨认笔录，证人何某、韩某、赵某、刘某、徐某、胡某、袁某、张某、周某均在见证人的见证下，从12张年龄相仿的女性免冠照片中辨认出柯桂菊就是向他们贩卖毒品的“二嫂”。以及柯桂菊辨认出赵某是在她手中买过毒品的人。11、搜查笔录，证实对柯桂菊家搜查，并搜查出疑似毒品红色颗粒物一颗。12、扣押物品清单、扣押决定书，证实2017年3月29日，公安机关被告人柯桂菊持有的疑似麻果60颗（红色58颗、绿色2颗）、疑似冰毒一袋（白色晶体）。13、、称重笔录，在见证人和被告人柯桂菊参与下，公安机关对扣押的疑似毒品进行称重，麻果5.62克、冰毒0.27克，共计5.89克。取样笔录，证实在见证人和被告人柯桂菊参与下，公安机关对疑似毒品进行取样、密封、盖章，并准备送鉴定中心鉴定。14、黄冈市公安司法鉴定中心检验报告，对公安机关送检的检材鉴定均含有甲基苯丙胺毒品成分。15、到案经过证实被告人柯桂菊系被抓获归案。16、户籍证明证实被告人柯桂菊的个人户籍信息。17、武汉市江岸区人民法院刑事判决书两份，证实被告人柯桂菊曾因犯贩卖毒品罪被判处刑罚情况。18、红安县公安局办案说明，证实在柯桂菊随身携带提包中搜出疑似麻果59颗、疑似冰毒一袋。并证实，在对柯桂菊住处搜查过程中，赵某敲门找柯桂菊，称是想买毒品的事实。19、被告人柯桂菊的供述和辩解：（2017年3月29日供述）毒品卖给过一个男子三次，在红安工业园区阳福社区开早餐店，店名“湖南米粉”，就在我家附近。第一次应当是2017年3月初的时候，他到我家门口，敲门说要买麻果，我问他怎么知道我有，他说是别人告诉他的。我问他要多少毒品，他说150元钱的，我就卖给他三颗麻果。第二次在大约过了一个礼拜左右，他又来敲我家门，问我有没有东西（毒品），我问他要多少，他说两颗红的（麻果），一点白的（冰），我就卖给他了。第三次是3月26、7号左右，他又来敲门，买了四颗麻果。我们都是现金交易，我通过我家侧面的窗户将毒品递给他。还有一个叫黑皮的人，微信号×××，我不知道叫什么，3月27日晚上通过微信转账，买了250元钱，五颗麻果。我也是通过我家窗户递给他的。还有些人我不认识，也不知道叫什么，他们都只是来买一、两次，每次都是一百至两百元钱。（2017年4月18日供述）我是2017年春节以后开始贩卖毒品的，是为赚钱供我自己吸食毒品。我卖给过四个人，其中有一个是在八里开米粉店的外地男子，就具体叫什么我不清楚，他在我手中买过三次，前后大概买了500元左右的毒品；再就是一个外号叫黑子的人，在我这里买过几遍，大概400元左右的麻果；还有两个男子也在我这里买过麻果，具体细节记不清了。本院认为，被告人被告人柯桂菊无视国法，多次向多人贩卖毒品，情节严重，其行为构成贩卖毒品罪。公诉机关指控成立，本院予以支持。被告人柯桂菊被判处有期徒刑刑罚执行完毕后，五年内再犯应当判处有期徒刑以上之罪，是累犯，应当从重处罚。被告人柯桂菊称没有卖毒品给那么多人的辩解意见，因有证人证言的合理陈述以及辨认笔录等证据证实其多次向多人贩卖毒品的事实，对被告人的该辩解意见本院不予采信。据此，依照《中华人民共和国刑法》第三百四十七条第四款之规定，判决如下：被告人柯桂菊犯贩卖毒品罪，判处有期徒刑五年六个月，并处罚金人民币5000元。罚金于本判决生效之日起十日内缴纳.（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年3月30日起至2022年9月29日止。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖北省黄冈市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判长　　李平辉审判员　　曾志斌审判员　　刘松涛二〇一七年九月十八日书记员　　余爱军 马 克 数 据 网"

text_8 = "湖南省长沙市天心区人民法院刑 事 判 决 书(2017)湘0103刑初504号公诉机关长沙市天心区人民检察院。被告人曹俊朝,男,1994年11月15日出生于湖南省双峰县。2017年6月5日被抓获，因涉嫌贩卖毒品罪,次日被长沙市公安局天心分局刑事拘留;于2017年7月11日经长沙市天心区人民检察院批准,次日被长沙市公安局天心分局执行逮捕。现羁押于长沙市第一看守所。辩护人任孝，湖南天地人律师事务所律师。被告人杨嘉,男,1996年1月11日出生于长沙市。2017年6月5日被抓获，因涉嫌贩卖毒品罪,次日被长沙市公安局天心分局刑事拘留;于2017年7月11日经长沙市天心区人民检察院批准,于次日被长沙市公安局天心分局执行逮捕。现羁押于长沙市第一看守所。辩护人代勇贤，湖南天地人律师事务所律师。长沙市天心区人民检察院以长天检诉刑诉（2017）520号起诉书指控被告人曹俊朝、杨嘉犯贩卖毒品罪，于2017年9月12日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。长沙市天心区人民检察院指派检察员焦焰出庭支持公诉，被告人曹俊朝及其辩护人任孝、被告人杨嘉及其辩护人代勇贤均到庭参加诉讼。现已审理终结。长沙市天心区人民检察院指控：2017年6月,被告人曹俊朝从他人处购买甲基苯丙胺(冰毒)、甲基苯丙胺片剂(俗称“麻古”),并安排被告人杨嘉寻找购买毒品的人员。2017年6月4日晚,被告人杨嘉的朋友“大姚”(另案处理)电话联系被告人杨嘉要购买毒品,被告人杨嘉告知被告人曹俊朝,被告人曹俊朝与“大姚”约定毒品交易价格、地点后,用卫生纸包装少量甲基苯丙胺(冰毒)和1小粒甲基苯丙胺片剂(俗称“麻古”)交给被告人杨嘉,被告人杨嘉骑被告人曹俊朝的摩托车至本市天心区南托岭纵鑫网吧门口与“大姚”安排来交易毒品的蔡某见面,被告人杨嘉将上述毒品以260元的价格贩卖给蔡某,并将收取的毒资260元交给被告人曹俊朝,被告人曹俊朝分给被告人杨嘉60元作为报酬。因“大姚”要求还要购买1套毒品,被告人曹俊朝将少量甲基苯丙胺(冰毒)和1小粒甲基苯丙胺片剂(俗称“麻古”)交给被告人杨嘉。2017年6月5日下午15时许,被告人杨嘉与蔡某电话联系后,至南托岭纵鑫网吧2楼准备再次贩卖毒品给蔡某时被公安民警查获,公安民警起获被告人杨嘉的0.32克白色晶体、0.06克红色药丸。当日晚,被告人杨嘉协助公安民警抓获被告人曹俊朝,公安民警从被告人曹俊朝身上起获0.53克白色晶体、0.1克红色药丸。经司法鉴定,从被告人曹俊朝、杨嘉处起获的白色晶体均含有甲基苯丙胺成分,起获的红色药丸均检出甲基苯丙胺、咖啡因成分。二被告人归案后均如实供述自己的罪行。该院认为，被告人曹俊朝、杨嘉违反国家关于毒品管理的规定，明知甲基苯丙胺、甲基苯丙胺片剂系国家禁止贩卖的毒品而予以非法贩卖，其行为已触犯《中华人民共和国刑法》第三百四十七条第一、四款，应当以贩卖毒品罪追究其刑事责任。本案系共同犯罪,被告人曹俊朝系主犯,适用《中华人民共和国刑法》第二十五条第一款、第二十六条第四款;被告人杨嘉系从犯,适用《中华人民共和国刑法》第二十五条第一款、第二十七条,具有法定从轻处罚情节。被告人杨嘉协助公安机关抓获同案犯,具有立功情节,适用《中华人民共和国刑法》第六十八条。该院为上述指控提供了相关证据，并建议本院对被告人曹俊朝、杨嘉犯贩卖毒品罪分别在有期徒刑一年以下与有期徒刑八个月以下予以量刑，并处罚金。被告人曹俊朝、杨嘉对公诉机关指控的事实当庭供认属实；对公诉人当庭出示和宣读的证据，表示没有异议。被告人曹俊朝的辩护人提出以下辩护意见：1、被告人曹俊朝归案后能如实供述自己的罪行，具有坦白情节，依法可以减轻处罚。2、本案中公诉机关指控的第一笔涉案毒品并未查货，没有直接证据证明该笔毒品的重量，根据疑罪从无和有利于被告人原则，对该笔指控应该从轻处罚；3、公诉机关指控的2017年6月5日杨嘉贩毒的行为，被告人曹俊朝并不知情，与被告人曹俊朝无关；指控的6月6日的贩毒行为，被告人曹俊朝并没有真实交易，且交易行为尚未完成。4、被告人曹俊朝系初犯，认罪态度好，且家庭责任较重，恳请法庭从轻处罚。被告人杨嘉的辩护人提出以下辩护意见：1、被告人杨嘉自愿认罪，具有坦白情节，依法可以从轻处罚。2、被告人杨嘉协助公安机关抓获同案犯，具有立功情节。3、被告人杨嘉贩卖的毒品数量极少，对社会和个人造成很小的危害。4、被告人杨嘉年龄不大，属青年人犯罪，是初犯，偶犯，量刑时应从教育挽救的角度出发，从轻量刑。经审理查明，公诉机关指控被告人曹俊朝、杨嘉犯罪的事实属实，情节没有出入，本院予以确认。上述事实，被告人曹俊朝、杨嘉在开庭审理过程中亦无异议，并有查获的毒品照片、手机通话详单、抓获经过、扣押决定书、扣押物品清单、户籍资料、证人蔡某的证言、被告人曹俊朝、杨嘉的供述和辩解、鉴定意见、搜查笔录、称量笔录、辨认笔录、视听资料、被告人曹俊朝与“大姚”的微信聊天记录、毒品称重同步录音录像、讯问同步录像光盘等证据证实，足以认定。本院认为，被告人曹俊朝、杨嘉违反国家关于毒品的管理规定，明知甲基苯丙胺、甲基苯丙胺片剂系国家禁止贩卖的毒品而予以非法贩卖，其行为已构成贩卖毒品罪。长沙市天心区人民检察院指控被告人曹俊朝、杨嘉犯贩卖毒品罪的事实及罪名成立，本院应对二被告人的行为依法予以惩处。被告人曹俊朝的辩护人提出公诉机关指控的2017年6月5日杨嘉贩毒的行为，被告人曹俊朝并不知情，与被告人曹俊朝无关的辩护意见，经查，依据被告人曹俊朝、杨嘉的供述以及二被告人与“大姚”之间的手机通话详单、微信聊天记录等证据，均能证实被告人曹俊朝对6月5日的杨嘉与购毒者交易毒品行为是知情和默认的，故辩护人提出的该辩护意见本院不予采纳。被告人曹俊朝的辩护人提出被告人曹俊朝在控制下交易毒品行为，未造成社会危害的辩护意见，本院在量刑时酌情予以考虑。本案系共同犯罪，被告人曹俊朝系主犯；被告人杨嘉系从犯，应当从轻处罚。被告人杨嘉协助公安机关抓获同案犯，具有立功情节，依法可以从轻处罚。被告人杨嘉的辩护人提出的此辩护意见，本院予以采纳。被告人曹俊朝、杨嘉归案后能如实供述自己的罪行，依法可以从轻处罚。二辩护人提出的此辩护意见，本院予以采纳。二辩护人提出被告人系初犯，认罪态度好，家庭和个人情况特殊的辩护意见，本院在量刑时予以考虑。据此，依照《中华人民共和国刑法》第三百四十七条第一、四款、第二十五条第一款、第二十六条第一、四款、第二十七条、第六十八条、第六十七条第三款之规定，判决如下：一、被告人曹俊朝犯贩卖毒品罪，判处有期徒刑九个月，并处罚金人民币三千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年6月5日起至2018年3月4日止。罚金限在本判决生效后十日内缴清。）二、被告人杨嘉犯贩卖毒品罪，判处拘役四个月，并处罚金人民币一千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年6月5日起至2017年10月4日止。罚金限在本判决生效后十日内缴清。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向湖南省长沙市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本两份。审判员　　马铁夫二〇一七年九月十八日书记员　　钟　真附相关法律条文：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：(一)走私、贩卖、运输、制造鸦片一千克以上、氯胺酮或者甲基苯丙胺五十克以上或者其他毒品数量大的;(二)走私、贩卖、运输、制造毒品集团的首要分子;(三)武装掩护走私、贩卖、运输、制造毒品的;(四)以暴力抗拒检查、拘留、逮捕，情节严重的;(五)参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、氯胺酮或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、氯胺酮或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金;情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第二十五条共同犯罪是指二人以上共同故意犯罪。二人以上共同过失犯罪，不以共同犯罪论处；应当负刑事责任的，按照他们所犯的罪分别处罚。第二十六条组织、领导犯罪集团进行犯罪活动的或者在共同犯罪中起主要作用的，是主犯。三人以上为共同实施犯罪而组成的较为固定的犯罪组织，是犯罪集团。对组织、领导犯罪集团的首要分子，按照集团所犯的全部罪行处罚。对于第三款规定以外的主犯，应当按照其所参与的或者组织、指挥的全部犯罪处罚。第二十七条在共同犯罪中起次要或者辅助作用的，是从犯。对于从犯，应当从轻、减轻处罚或者免除处罚。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第六十八条犯罪分子有揭发他人犯罪行为，查证属实的，或者提供重要线索，从而得以侦破其他案件等立功表现的，可以从轻或者减轻处罚;有重大立功表现的，可以减轻或者免除处罚。 百度搜索“马 克 数 据 网”"

text_9 = "广东省珠海市香洲区人民法院刑 事 判 决 书（2017）粤0402刑初1011号公诉机关珠海市香洲区人民检察院。被告人陈茂鑫，男，1979年10月13日出生，汉族，初中文化，户籍地广东省徐闻县，因本案于2017年6月27日被羁押，次日被刑事拘留，同年7月18日被逮捕。现羁押于珠海市第一看守所。珠海市香洲区人民检察院以珠香检公诉刑诉[2017]1051号起诉书指控被告人陈茂鑫犯贩卖毒品罪，于2017年8月10日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。珠海市香洲区人民检察院指派检察员姚少锋出庭支持公诉，被告人陈茂鑫到庭参加了诉讼。本案现已审理终结。经审理查明，2017年6月27日，被告人陈茂鑫在珠海市香洲区南屏镇广生巴士站附近，将一包甲基苯丙胺（冰毒）以200元（人民币，下同）的价格贩卖给吸毒人员陈某1。同年6月27日20时许，被告人陈茂鑫通过电话与陈某1联系，在珠海市香洲区南屏镇广生巴士站附近，将一包净重0.57克的甲基苯丙胺以300元的价格贩卖给陈某1后，被民警当场抓获。民警现场从被告人陈茂鑫身上查获用于联系毒品交易的型号为JC－V9的黑色手机一部，毒资现金300元；从陈某1身上查获甲基苯丙胺一小包，净重0.57克。另查明，被告人陈茂鑫为初中文化，珠海市香洲区人民检察院珠香检公诉刑诉[2017]1051号起诉书认定其为小学文化，有误，予以纠正。上述事实，被告人陈茂鑫在开庭审理过程中亦无异议，且有经过庭审举证、质证，本院予以确认的证人陈某1和陈某2的证言及辨认笔录，抓获经过，搜查笔录、扣押清单，提取笔录、称量取样笔录，检定证书、说明及结果，调取证据通知书、开户资料、通话清单，户籍资料，前科材料，现场检测报告书，情况说明，珠公司化〔2017〕599号理化检验报告书，鉴定意见通知书，现场照片，手机一部等证据证实，足以认定。本院认为，被告人陈茂鑫贩卖毒品，其行为已经触犯我国刑律，构成贩卖毒品罪。公诉机关指控被告人陈茂鑫的犯罪事实清楚，证据确实、充分，指控罪名成立。被告人陈茂鑫归案后如实供述自己罪行，依法可以从轻处罚。依照《中华人民共和国刑法》第三百四十七条第四款、第六十七条第三款、第六十四条的规定，判决如下：一、被告人陈茂鑫犯贩卖毒品罪，判处有期徒刑十个月，并处罚金人民币一千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年6月27日起至2018年4月26日止。罚金自判决生效之日起一个月内缴纳。）二、扣押在案的型号为JC－V9的黑色手机一部和甲基苯丙胺0.57克，均予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向广东省珠海市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审　判　长　　蔡超明审　判　员　　彭　帅人民陪审员　　谢冬霖二〇一七年九月五日书　记　员　　冼宇新林链附相关法条：《中华人民共和国刑法》第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。 更多数据：搜索“马克数据网”来源：www.macrodatas.cn"

text_10 = "福建省永安市人民法院刑 事 判 决 书（2017）闽0481刑初212号公诉机关永安市人民检察院。被告人郭小平（绰号“锅嫖”），男，1986年9月23日出生，汉族，初中文化，无业，现住福建省永安市（户籍所在地：江西省瑞金市）。曾因犯开设赌场罪，于2013年7月25日被本院判处拘役五个月，并处罚金人民币10000元,于2013年8月23日刑满释放。因涉嫌贩卖毒品罪，于2017年4月1日被刑事拘留，同年4月11日被逮捕。现羁押于永安市看守所。永安市人民检察院以永检公刑诉﹝2017﹞217号起诉书指控被告人郭小平犯贩卖毒品罪，于2017年5月16日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。永安市人民检察院指派检察员罗奕炫出庭支持公诉，被告人郭小平到庭参加诉讼。现已审理终结。永安市人民检察院指控：2016年8月18日21时许，被告人郭小平在永安市名流财富广场8008KTV楼下以人民币300元的价格将1.8克毒品氯胺酮（俗称“K粉”）卖给吸毒人员赖某。2017年4月1日，被告人郭小平主动到永安市公安局投案。上述事实，被告人郭小平在开庭审理过程中亦无异议，且有公诉机关提供，并经庭审举证、质证的下列证据予以证实，具有合法性、真实性和关联性，本院予以确认。1.人口基本信息表、违法犯罪经历查询情况表、刑事判决书、释放证明；2.人体生物样本检测提取笔录、现场检测报告书；3.行政处罚决定书；4.在逃人员登记信息表、到案经过、情况说明；5.证人赖某的证言；6.被告人郭小平的供述和辩解；7.辨认笔录、辨认现场照片、检测照片等证据予以证实，足以认定。本院认为，被告人郭小平明知是氯胺酮而予以贩卖，其行为构成贩卖毒品罪。公诉机关指控的罪名成立。被告人郭小平曾因故意犯罪被判处刑罚，刑满释放后不思悔改再次犯罪，可酌情从重处罚。被告人郭小平犯罪以后自动投案，如实供述自己的犯罪事实，属自首，可以从轻处罚。据此，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第六十七条第一款之规定，判决如下：被告人郭小平犯贩卖毒品罪，判处拘役四个月，并处罚金人民币1000元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日。即自2017年4月1日起至2017年7月31日止。罚金应于本判决发生法律效力之次日起十日内缴纳。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向福建省三明市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判员　　崔璐二〇一七年五月二十七日书记员　　叶云PAGE 来自：www.macrodatas.cn"

text_11 = "甘肃省榆中县人民法院刑 事 判 决 书（2017）甘0123刑初187号公诉机关榆中县人民检察院。被告人周怀庆，男，汉族，1962年8月16日出生，居民，小学文化程度，甘肃省榆中县人，农民，住榆中县。2013年9月17日因犯贩卖毒品罪被兰州市城关区人民法院判处有期徒刑一年，并处罚金3000元。2017年1月4日因涉嫌贩卖毒品罪被榆中县公安局刑事拘留，2017年1月26日经榆中县人民检察院批准被逮捕。现羁押于榆中县看守所。榆中县人民检察院以榆检公诉刑诉〔2017〕153号起诉书指控被告人周怀庆犯贩卖毒品罪，于2017年5月11日向本院提起公诉。本院审查后于同日立案并依法组成合议庭，适用简易程序，于2017年5月18日依法公开开庭审理了本案。榆中县人民检察院指派代理检察员王娅妮出庭支持公诉。被告人周怀庆到庭参加了诉讼。现已审理终结。经审理查明：2016年5月份，张加利代武国华从被告人周怀庆手中购买了400元的海洛因；同年5月份，周怀庆乘坐冯康成的车从榆中县来紫堡乡到兰州市东岗镇买菜，回来后用海洛因抵车费；同年5月份，周怀庆在找张加红干活后给其一小包海洛因抵工钱；同年七月份，周怀庆以500元的价格向张加利贩卖海洛因一小包。上述事实，被告人周怀庆在开庭审理过程中亦无异议，并有案件来源、常住人口基本信息查询、违法犯罪记录查询情况说明、抓获经过、搜查笔录、扣押清单、通话记录、兰公（城）强戒决字{2017}190号强制隔离戒毒决定书、刑事判决书、刑满释放人员通知书，辨认笔录，证人张加利、武国华、张加红、冯康成、李刚的证言，被告人周怀庆的供述与辩解供述等证据证实，足以认定。本院认为，被告人周怀庆目无国法，向他人贩卖毒品海洛因，其行为已触犯刑律，构成贩卖毒品罪，依法应予惩处。公诉机关的指控，事实清楚，证据确实、充分，适用法律正确，罪名成立，应予以支持。本案中被告人周怀庆曾因犯贩卖毒品罪被判处有期徒刑以上刑罚，在刑罚执行完毕后五年内又多次向他人贩卖毒品，故意犯罪的，属累犯，毒品再犯，依法应从重处罚，但其在庭审中自愿认罪，有悔罪表现，可酌情从轻处罚。公诉机关关于对被告人周怀庆判处有期徒刑三年六个月至四年六个月，并处罚金的量刑建议符合法律规定，应予采纳。本院为严肃国法，维护社会管理秩序，打击毒品犯罪，依据《中华人民共和国刑法》第三百四十七条第一款、第四款、第三百五十六条、第五十二条、第五十三条、第六十四条、第六十五条之规定，判决如下：一、被告人周怀庆犯贩卖毒品罪，判处有期徒刑四年，并处罚金5000元（限于判决生效后三十日内缴纳）。二、随案移送的电子秤一台，予以没收。上缴国库。（刑期从判决执行之日起计算，判决执行之前先行羁押的，羁押一日折抵刑期一日。即自2017年1月4日起至2021年1月3日止。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向甘肃省兰州市中级人民法院提出上诉，书面上诉的，应当提交上诉状正本一份，副本两份。审　判　长　　王在娟代理审判员　　陈　昀人民陪审员　　金永江二〇一七年五月十八日书　记　员　　吕瑞娟 关注微信公众号“马克数据网”"

text_12 = "甘肃省定西市安定区人民法院刑 事 判 决 书（2017）甘1102刑初86号公诉机关定西市安定区人民检察院。被告人马麻吾乃，男，回族，生于1978年2月1日，甘肃省广河县人，文盲，农民，住广河县。无前科。因涉嫌犯非法持有毒品罪于2016年4月6日被定西市安定区公安局刑事拘留，同年4月13日被定西市安定区公安局取保候审，2017年2月26日被定西市安定区公安局以涉嫌犯运输毒品罪刑事拘留，2017年3月3日被执行逮捕。现羁押于定西市安定区看守所。辩护人马红平，甘肃王力律师事务所律师。定西市安定区人民检察院以安检刑诉【2017】77号起诉书指控被告人马麻吾乃犯非法持有毒品罪，于2017年3月23日向本院提起公诉。本院审查立案后，依法组成合议庭，于2017年4月26日公开开庭审理了本案。定西市安定区人民检察院指派检察员李相国出庭支持公诉，被告人马麻吾乃及其辩护人马红平到庭参加诉讼。现已审理终结。定西市安定区人民检察院指控：2016年4月6日，定西市安定区公安局民警在定西市安定区永定路宏通苑小区26号楼2单元门口将携带毒品的被告人马麻吾乃抓获，当场从其身上查获毒品海洛因49.87克。针对指控的犯罪事实,公诉机关提请审判长同意,当庭宣读并出示了受案登记表，立案决定书，被告人马麻吾乃供述，证人康继发证言，查获笔录，称量笔录，理化检验鉴定报告，现场检测报告书，机主信息及通话详单，机主信息查询，抓获经过，被告人马麻吾乃户籍证明等证据。公诉机关认为,被告人马麻吾乃非法持有毒品海洛因49.87克,其行为触犯了《中华人民共和国刑法》第三百四十八条之规定，应当以非法持有毒品罪追究其刑事责任。被告人马麻吾乃对公诉机关指控的犯罪事实供认不讳。当庭认罪，请求从轻处罚。辩护人辩称：公安机关在毒品的提取、扣押、称量、取样程序等方面存在不符合法律规定的情况，即：没有毒品提取笔录、封装笔录，毒品是否封装以及封装过程没有任何反映，毒品称量笔录没有见证人签名，没有法定计量检定机构出具的计量检定证书，也没有关于称量过程的记录和对称量的主要过程进行拍照或录像，没有毒品鉴定的取样笔录，毒品检验的检材选取或抽取是否合法无法确认，毒品扣押清单中只有一名侦查人员签名；被告人持有的毒品尚未流入社会，其行为的社会危害性相对较小；被告人系初犯，认罪态度较好。建议对被告人宽大处理。经审理查明：2016年4月6日，定西市安定区公安局民警在定西市安定区永定路宏通苑小区26号楼2单元门口将被告人马麻吾乃抓获，当场从其身上查获其非法持有的毒品海洛因49.87克。认定上述事实的证据有：1、受案登记表，立案决定书，证明定西市安定区公安局禁毒大队民警于2016年4月6日在工作中发现被告人马麻吾乃涉嫌非法持有毒品，予以立案侦查。2、被告人马麻吾乃供述，证明2016年4月6日，其在临洮县康家崖镇从一个人跟前以每克300元的价格购买了50克毒品海洛因后，驾驶自己的白色东风日产车携带该毒品到定西市安定区找曾一起与其赌博欠其15000元钱的一个定西人时，在定西市安定区盘旋路旁边的一个小区里被警察抓获，警察从其所穿外衣左侧内口袋里查获毒品海洛因49.87克。3、抓获经过，查获笔录，称量笔录，理化检验鉴定报告，扣押决定书，扣押清单，证明定西市安定区公安局禁毒大队民警于2016年4月6日19时许在定西市安定区永定路宏通苑小区26号楼2单元门口将被告人马麻吾乃抓获，当场从其身上查获其非法持有的毒品海洛因49.87克，予以扣押，并扣押其持有的白色VIVO手机一部、黑色三星直板手机一部、红色三星上翻盖手机一部。4、现场检测报告书，证明被告人马麻吾乃于2016年4月6日的尿液经吗啡、甲基苯丙胺联合检测试剂检测，结果呈阴性。5、被告人马麻吾乃户籍证明，证明被告人马麻吾乃姓名、籍贯、出生日期、住址等身份信息。上述证据，经庭审质证，均取证合法，各证据之间能够相互印证证实本院审理查明的案件事实，本院确认其证明效力。公诉机关当庭宣读出示的证人康继发证言，手机通话清单，机主信息查询，对本案不具有证明作用，证明效力不予确认。辩护人提交的被告人马麻吾乃的出院证明书及其母亲马某某某、妻子克某某的低保证对本案不具有证明作用，证明效力不予确认。本院认为，被告人马麻吾乃违反国家对毒品的管理制度，非法持有毒品海洛因49.87克,其行为构成非法持有毒品罪。被告人马麻吾乃当庭认罪，酌情从轻处罚。定西市安定区人民检察院指控被告人马麻吾乃犯非法持有毒品罪的事实清楚，证据确实、充分，予以认定。被告人马麻吾乃请求对其从轻处罚的量刑辩护意见成立，予以采纳。辩护人的辩护意见成立，予以采纳。公安机关侦办本案过程中，虽然在毒品的提取、扣押、称量、取样程序等方面存在瑕疵，但不影响本案事实的认定、定性。根据被告人马麻吾乃犯罪的性质、情节、社会危害程度，依照《中华人民共和国刑法》第三百四十八条，第五十二条，第五十三条，第六十四条之规定，判决如下：一、被告人马麻吾乃犯非法持有毒品罪，判处有期徒刑二年，并处罚金2000元。（刑期从判决执行之日起计算；判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年2月26日起至2019年2月18日止。罚金限判决生效后立即缴纳）二、随案移交的手机，发还被告人马麻吾乃。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向甘肃省定西市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本一份。审　判　长　　王建军人民陪审员　　李立功人民陪审员　　何晓燕二〇一七年五月二十二日书　记　员　　丁文斌 微信公众号“马克 数据网”"

text_13 = "湖南省洞口县人民法院刑 事 判 决 书（2017）湘0525刑初18号公诉机关湖南省洞口县人民检察院。被告人王也，男,1980年10月9日出生于湖南省洞口县，汉族,高中文化，石江镇农合办职工,住洞口县。因涉嫌贩卖毒品犯罪于2016年9月8日被刑事拘留，同年10月12日被逮捕。现押洞口县看守所。被告人米敏，女,1986年3月3日出生于湖南省洞口县，汉族,初中文化,无业，住洞口县。因涉嫌贩卖毒品犯罪于2016年11月25日被指定居所监视居住于邵阳市强制隔离戒毒所，2017年3月8日被逮捕，现押邵阳市看守所。湖南省洞口县人民检察院以洞检公诉刑诉（2016）238号起诉书指控被告人王也、米敏犯贩卖毒品罪，于2017年1月20日向本院提起公诉。本院受理后，依法组成合议庭，公开开庭审理了本案。洞口县人民检察院指派代检察员周琼出庭支持公诉，被告人王也、米敏到庭参加了诉讼。现已审理终结。洞口县人民检察院指控，2016年8月至9月8日间，被告人王也贩卖毒品冰毒6次，贩卖毒品数量7．88克；被告人米敏与被告人王也共同贩卖毒品冰毒3次，贩卖毒品数量4．4克。为证明指控的事实，公诉机关当庭宣读并提交了两被告人的供述；提取及称量笔录、扣押决定书、扣押物品清单；手机微信转账记录截图、手机通话详单；现场检测报告；证人张某、彭某、肖某、欧某的证言等。认为两被告人违反国家毒品管理规定，多次向他人出售毒品的行为已触犯刑律，应追究刑事责任，请求依法判处。被告人王也辩称，他向张某贩卖毒品冰毒一次属实，其余都不是事实；微信转账记录是他们之间互相转发的，并不能证明是毒资。被告人米敏对上述指控事实没有异议。但在庭审提出毒品是王也的，吸毒人员买毒品时毒资也是交给王也，她只是代替王也发出毒品。经审理查明：一、2016年8月份的某天，吸毒人员彭某在洞口县城“如家”宾馆被告人王也、米敏所开的房间内，以人民币500元向被告人王也、米敏购买冰毒约3克；几天后彭某又在“如家”宾馆被告人王也、米敏所开的房间内，以人民币200元向被告人王也、米敏购买冰毒约1克。上述事实，有下列经过庭审质证并且查证属实的证据证明：1、被告人米敏的供述证明，2016年8月份的一天“梅子”（彭某）买了500元钱的货，她用这500元钱交了房费和给王也充了游戏币；又过了几天，“梅子”又找到王也买了200元钱的货，她用这200元钱给王也充了游戏币。2、证人彭某证明，今年8月份在如家宾馆从米敏手上买了500元钱；又过了几天（8月中旬）也是在如家宾馆从米敏手上买了200元钱。二、2016年8月份的某天，吸毒人员肖某在被告人王也租住的“依星级”宾馆一楼的房间内，以人民币400元向被告人王也购买冰毒约2克。上述事实，有下列经过庭审质证并且查证属实的证据证明：1、被告人王也的供述，供述2016年8月份的一天，一个叫肖某的年轻男子联系要货，在依星级宾馆一楼他给了肖某约2克冰毒，肖某给了他400元现金。2、证人肖某证明，2016年8月份的一天，在依星级宾馆一楼，他给了王也400元现金，王也给了他2克冰毒。三、2016年8月中旬的某天，吸毒人员张某通过微信转账的方式转账100元给被告人王也，然后在“如家”宾馆407房间将其已转账给被告人王也100元的事情告诉被告人米敏，从米敏手中拿到冰毒约0．4克。四、2016年9月6日晚上，吸毒人员张某在被告人王也租住的“依星级”宾馆一楼的房间内，以人民币100元向被告人王也购买冰毒约0．5克。五、2016年9月8日5时许，吸毒人员张某再次来到被告人王也租住的“依星级”宾馆一楼的房间内，以人民币100元向被告人王也购买冰毒约0．4克。2016年9月8日，公安机关在被告人王也租住的“依星级”宾馆一楼的房间内搜缴冰毒疑似物净重0．58克，经理化鉴定，该搜缴物含有甲基苯丙胺成分。上述事实，有下列经过庭审质证并且查证属实的证据证明：1、被告人王也的供述，供述2016年9月6日在依星级宾馆一楼，他给了张某0．5克冰毒，张某给了他100元钱；今天凌晨张某问有没有冰毒，他给了张某大概0．5克冰毒，张某用微信红包转给他100元。2、被告人米敏的供述，供述张某多次到王也处买冰毒的，记得比较清楚8月份在如家宾馆一次，9月6日和9月8日凌晨在依星级宾馆各一次。3、证人张某证明，2016年9月6日在依星级宾馆他给了王也100元现金，王也给了他大概0．5克冰毒；9月8日凌晨在依星级宾馆他给王也转了100元，王也给了他大概0．4克冰毒；2016年8月中旬，他用微信转了100元给王也后，到如家宾馆米敏手拿了大概0．4克冰毒。4、证人彭某证明，2016年9月6日在依星级宾馆，“蒙古”（张某）给了王也100元钱，王也给了“蒙古”一小包冰毒；王也被抓的那天凌晨，王也用一个烟盒外的塑料袋装了一点冰毒给“蒙古”，“蒙古”讲身上现金不够，用微信转账。5、证人欧某证明，“蒙古佬”用他的手机登录“蒙古佬”自己的微信号给王也转了100元。6、提取笔录、扣押物品清单、称量笔录、微信转账记录截图、通话详单等，证明在被告人王也所开的房间内搜缴冰毒0．58克及被告人王也与张某的微信转账和通话记录等。7、现场检测报告、理化检验鉴定报告，证明被告人王也的尿液检测结果为阳性和在被告人王也所开的房间内搜缴的0．58克物品中含有甲基苯丙胺成分。8、户籍信息，证明了两被告人的身份情况。本院认为：被告人王也、米敏违反国家毒品管理规定，多次向他人贩卖毒品，情节严重，其行为均已构成贩卖毒品罪。公诉机关指控的犯罪事实清楚，证据确实充分，罪名成立。在共同贩卖毒品犯罪中，被告人王也起主要作用，是主犯，应当按照其所参与的全部犯罪处罚；被告人米敏起协助作用，系从犯，应当从轻处罚；归案后，被告人米敏能如实供述自己的犯罪事实，是坦白，可以从轻处罚。公诉机关指控被告人米敏是主犯，经查，被告人米敏所贩卖的毒品是被告人王也的，毒资也是王也收取，被告人米敏只是帮助王也发出毒品，所起的是协助作用，故对公诉机关指控被告人米敏是主犯的指控，本院不予支持。据此,对被告人王也适用《中华人民共和国刑法》第三百四十七条第一款、第四款、第七款，第二十六条第一款、第四款，第五十二条的规定；对被告人米敏适用《中华人民共和国刑法》第三百四十七条第一款、第四款、第七款，第二十七条、第五十二条、第六十七条第三款的规定，判决如下：一、被告人王也犯贩卖毒品罪，判处有期徒刑四年，并处罚金人民币五千元（已缴纳）。(刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2016年9月8日起至2020年9月7日止。)二、被告人米敏犯贩卖毒品罪，判处有期徒刑二年，并处罚金人民币一千元（已缴纳）。(刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2016年11月25日起至2018年11月24日止。)如不服本判决,可在接到判决书的第二日起十日内,通过本院或者直接向湖南省邵阳市中级人民法院提出上诉。书面上诉的,应当提交上诉状正本一份、副本二份。审判长　　谢建华审判员　　尹显刚审判员　　曾晓勋二〇一七年五月十七日书记员　　贺　勃附相关法条：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第二十六条组织、领导犯罪集团进行犯罪活动的或者在共同犯罪中起主要作用的，是主犯。三人以上为共同实施犯罪而组成的较为固定的犯罪组织，是犯罪集团。对组织、领导犯罪集团的首要分子，按照集团所犯的全部罪行处罚。对于第三款规定以外的主犯，应当按照其所参与的或者组织、指挥的全部犯罪处罚。第二十七条在共同犯罪中起次要或者辅助作用的，是从犯。对于从犯，应当从轻、减轻处罚或者免除处罚。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。 来自：www.macrodatas.cn"

text_14 = "广东省茂名市电白区人民法院刑 事 判 决 书（2017）粤0904刑初219号公诉机关茂名市电白区人民检察院。被告人张学燚（绰号煲仔），男，1994年8月5日出生，汉族，初中文化，户籍所在地广东省茂名市电白区，住电白区，2016年3月1日因本案被茂名市公安局电白分局取保候审，2017年3月1日因涉嫌贩卖毒品罪被逮捕。现押于茂名市电白区看守所。茂名市电白区人民检察院以茂电检诉刑诉〔2017〕166号起诉书指控被告人张学燚犯贩卖毒品罪，于2017年3月22日向本院提起公诉。本院依法组成合议庭，公开开庭审理了本案。茂名市电白区人民检察院指派代理检察员杨永诚出庭支持公诉，被告人张学燚到庭参加诉讼。现已审理终结。茂名市电白区人民检察院指控，被告人张学燚从2014年2月份开始贩卖毒品。张学燚从其上家“大眼仔”处购买毒品，然后用透明塑料袋包装好每包为50元至100元不等。2014年4月20日下午，张学燚在位于陈村镇水连坡村以50元人民币价格贩卖一小包毒品冰毒给吸毒人员林某；2014年7月8日下午，张学燚在位于陈村镇水连坡村三叉路口处以50元人民币价格贩卖一小包毒品冰毒给林某。2014年6月20日下午，张学燚在位于水东镇东苑宾馆门口处以100元人民币的价格贩卖一小包毒品冰毒给陈某2。2014年7月11日上午9时许，张学燚在位于陈村镇水连坡村边处以50元价格贩卖一小包海洛因给钟某。2014年6月中旬一天，张学燚在陈村镇陈村村处以50元人民币价格贩卖一小包冰毒给陈某1。2014年7月11日，民警在陈村镇盐灶后村抓获张学燚并在其身上缴获白色结晶体1小袋。经检验，该白色结晶体1袋检见毒品甲基苯丙胺成分，净重为0.49克。就上述指控，公诉机关向本院提供了相关证据，并认为被告人张学燚违反国家毒品管理法规，多次向多人贩卖毒品，其行为触犯了《中华人民共和国刑法》第三百四十七条之规定，犯罪事实清楚，证据确实、充分，应当以贩卖毒品罪追究其刑事责任。提请本院依法判处。被告人张学燚辩解没有贩卖过毒品海洛因给钟某，对其他指控没有异议。经审理查明，2014年4月20日下午，被告人张学燚在电白区陈村镇水连坡村以50元人民币价格贩卖1小包毒品冰毒给吸毒人员林某；2014年7月8日下午，张学燚在陈村镇水连坡村三叉路口处以50元人民币价格贩卖1小包毒品冰毒给林某。2014年6月20日下午，张学燚在水东镇东苑宾馆门口处以100元人民币的价格贩卖1小包毒品冰毒给陈某2。2014年6月中旬一天，张学燚在陈村镇陈村村处以50元人民币价格贩卖1小包冰毒给陈某1。2014年7月11日，民警在陈村镇盐灶后村查获张学燚，并缴获白色结晶体1小包。经检验，该白色结晶体1小包检见毒品甲基苯丙胺成分，净重0.49克。另查明，被告人张学燚在2014年7月11日被带回公安机关协助调查后离开，后于2016年3月1日在其亲属的陪同下到公安机关投案，并如实供述其贩卖毒品的犯罪事实。以上事实，有以下证据证实：（一）书证、物证1、受案登记表、立案决定书。证明案件的来源及立案情况。2、抓获经过、到案经过。证明张学燚于2014年7月11日曾被带回派出所协助调查；2016年3月1日张学燚在亲属的陪同下到公安机关投案。3、户籍资料。证明张学燚于1994年8月5日出生，案发时已达到负完全刑事责任年龄。4、扣押决定书、扣押清单。证明抓获张学燚时缴获疑似毒品物质1小包、手机1部、SIM卡1张。5、在逃人员登记信息表。证明张学燚曾因本案在逃。6、嫌疑人违法犯罪记录查询登记表。证明张学燚在本案之前无违法犯罪记录。7、行政处罚决定书、强制隔离戒毒决定书。证明证人林某、钟某、陈某1、陈某2是吸毒人员。（二）被告人张学燚供述及辨认笔录主要内容：我花名叫煲仔，我来投案自首。我自2014年1月不打工后，回到电白区陈村镇山水连坡村家中。同年2月份，我染上了吸毒恶习，由于无业加上家庭困难，我就产生了贩毒同时满足吸毒的念头。于是从2014年2月份开始，我从电白区旦场镇一名叫大眼仔那里购买毒品来贩卖。有时是我先从吸毒人员手上拿到钱后，再去找大眼仔买毒品回来给吸毒仔，我也蹭来吸食。我从大眼仔买来的毒品，包装是白色透明的小塑料袋，1袋2份，重量不清楚。价格一般是50元一小袋买来，卖给其他人便贵20到50元。我从中赚20到50元一次。我贩卖过给好几个人，但具体姓名不记得了，我只认识林某、陈某1、陈某2。他们几个都是以电话与我联系，约好交易地点，然后一手交钱一手交毒品。我一共贩卖二次毒品给陈某1，地点是陈村镇陈村村，时间不记得了。贩卖一次给陈某2，是在2014年6月20日是在水东镇水东圩交易冰毒100元。贩卖二次给林某，2014年4月份一日下午，在陈村镇水连坡村路口卖50元冰毒给他；第二次是7月份，我们在陈村镇水连坡三角路口处交易。不记得是否贩卖过毒品给钟某。我贩卖毒品没有牟利，所得都用来自己吸毒了。2014年7月11日被抓获时，因害怕，所以我没有如实交待犯罪事实。通过照片，张学燚辨认出向其购买过毒品的吸毒人员陈某2、陈某1、林某。（三）证人证言及辨认笔录1、陈某2证言及辨认笔录主要内容：我吸食的毒品是从陈村镇水连坡村张学燚处购买来的。我一共向张学燚购买两次毒品冰毒。第一次是2014年6月20日下午，我打电话给张学燚，在水东镇东苑宾馆门口处向张学燚购买100元冰毒，用透明塑料包装着的一小包；第二次是2014年7月11日，约好他送150元冰毒到水东镇三角圩盈通旅社门口，我还没有接到货就被抓获了。张学燚手机号是130××××2615。陈某2辨认出照片中1号男子是贩卖毒品给其的人（张学燚）。2、林某证言及辨认笔录主要内容：我的毒品是通过电话向一个叫“煲仔”的男子购买的。我向煲仔购买的是麻果，第一次是2014年4月20日在水连坡村边向他购买50元，第二次是2014年7月8日在水连坡三叉路口向他购买50元，都是用透明塑料包装着的一小包。煲仔的电话是130××××2615。我是用158××××0255联系他的。林某辨认出照片中3号是贩卖毒品给其的人（张学燚）。3、陈某1证言及辨认笔录主要内容：我是2014年6月14日被抓获的。公安在我房间搜查到的一小包白色晶体是冰毒，是我朋友洋鬼打电话给水连坡豹仔取的货，在我房间吸食后剩下的，他今天2时许离开我家回去了。豹仔是贩卖毒品的。我一般都是电话联系豹仔，他亲自开摩托车送一些冰毒到我家里，我们一手交钱，一手交货，每次都是购买50元人民币冰毒（麻果）。一般都是晚上交易。陈某1辨认出照片中9号是贩卖毒品给其的人（张学燚）。4、钟某证言及辨认笔录主要内容：我是2014年7月13日被传唤的，我吸食的毒品吗啡（白粉）是向陈村镇水连坡村一名叫煲仔的男子购买的。我直接到水连坡村附近找他购买，每次都是购买30元，他都是用塑料吸管包装着一小包。我自行到陈村镇水连坡村中间转几转他就会出现与我交易毒品。钟某辨认出照片中4号是贩卖毒品给其的人（张学燚）。5、邓某证言主要内容：2014年7月11日14时许，我在陈村镇那行盐灶后的大路处遇到张学燚并交谈，民警刚好巡逻经过，我看到民警在那条大路边草丛中捡起一小纸包东西，后民警将我们带回问话。这小纸包东西不是我的，不知道是谁的。（四）鉴定文书。经广东省茂名市公安司法鉴定中心鉴定，从张学燚处缴获的1小包疑似毒品物检见毒品甲基苯丙胺成分，重量为0.49克。（五）通话记录清单。证明被告人张学燚持有的手机130××××2615与陈某2持有的手机134××××1734、林某持有的电话158××××0255均有通话记录。（六）现场勘验笔录、现场图。证明案发现场位置及现场概况。上述证据均由公诉机关提供，来源合法，并经庭审公开举证、质证，可作为定案的依据。本院认为，被告人张学燚违反国家毒品管理规定，多次向多人贩卖毒品甲基苯丙胺，其行为已构成贩卖毒品罪，应予刑罚，依法应处三年以上七年以下有期徒刑，并处罚金。公诉机关指控被告人张学燚犯贩卖毒品罪的事实清楚，证据确实、充分，予以支持。张学燚辩解没有贩卖过毒品海洛因给钟某，经查，指控张学燚于2014年7月11日以50元价格贩卖1小包海洛因给钟某，虽有钟某指证，但据钟某证言，其是向张学燚购买吗啡（白粉），每次30元，并没有讲到2014年7月11日向张学燚购买50元海洛因，张学燚原供述也没有明确过有贩卖海洛因给钟某，所以该指控证据不充分，不予支持，张学燚辩解成立，予以采纳。被告人张学燚自动投案，并能够如实供述其犯罪行为，是自首，可以从轻或者减轻处罚。根据被告人张学燚的犯罪事实、犯罪情节、社会危害性和认罪态度，依法对张学燚减轻处罚。依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第六十七条第一款、第五十二条、第五十三条、第六十四条及《最高人民法院关于审理毒品犯罪案件适用法律若干问题的解释》第四条第一款第（一）项的规定，判决如下：一、被告人张学燚犯贩卖毒品罪，判处有期徒刑二年六个月，并处罚金人民币5000元；（刑期自判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年3月1日起至2019年8月31日止。罚金限于本判决发生法律效力之次日起十日内向本院缴付完毕。）二、缴获的毒品0.49克、SIM卡1张，予以没收，由公安机关依法处置；缴获的手机1部，予以没收，上缴获国库。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向广东省茂名市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审　判　长　　冼海英人民陪审员　　赖　就人民陪审员　　林国平二〇一七年五月九日书　记　员　　钟瑞城附相关法律条文：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第五十二条判处罚金，应当根据犯罪情节决定罚金数额。第五十三条罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。如果由于遭遇不能抗拒的灾祸缴纳确实有困难的，可以酌情减少或者免除。第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚;因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。最高人民法院关于审理毒品犯罪案件适用法律若干问题的解释法释[2016]8号第四条走私、贩卖、运输、制造毒品，具有下列情形之一的，应当认定为刑法第三百四十七条第四款规定的“情节严重”：（一）向多人贩卖毒品或者多次走私、贩卖、运输、制造毒品的；（二）在戒毒场所、监管场所贩卖毒品的；（三）向在校学生贩卖毒品的；（四）组织、利用残疾人、严重疾病患者、怀孕或者正在哺乳自己婴儿的妇女走私、贩卖、运输、制造毒品的；（五）国家工作人员走私、贩卖、运输、制造毒品的；（六）其他情节严重的情形。 关注微信公众号“马克数据网”"

text_15 = "吉林省松原市宁江区人民法院刑 事 判 决 书（2017）吉0702刑初97号公诉机关松原市宁江区人民检察院。被告人邱海（曾用名邱猛），男，捕前扶余市。曾因吸食毒品，于2012年3月1日被松原市公安局宁江一分局行政拘留十五日；又因吸毒、容留他人吸毒，于2014年2月23日被松原市公安局宁江一分局行政拘留二十日；又因吸毒，于2014年3月12日被松原市公安局宁江一分局强制隔离戒毒二年。曾因犯容留他人吸毒罪，于2011年4月18日被宁江区人民法院判处有期徒刑六个月，并处罚金5000元，于2011年8月17日刑满释放。又因吸毒，于2016年8月13日被松原市公安局宁江二分局强制隔离戒毒二年。因涉嫌贩卖毒品，于2016年8月15日被松原市公安局宁江二分局刑事拘留；因涉嫌犯贩卖毒品罪，于2016年8月26日由松原市公安局宁江二分局执行逮捕。现羁押于松原市看守所。松原市宁江区人民检察院以宁检公诉刑诉﹝2017﹞67号起诉书，指控被告人邱海犯贩卖毒品罪一案，于2017年2月20日向本院提起公诉。本院受理后，依法组成合议庭，于2017年3月9日、2017年4月26日公开开庭审理了本案。松原市宁江区人民检察院指派代理检察员王超出庭支持公诉，被告人邱海到庭参加了诉讼。现已审理终结。公诉机关指控，2016年7月某日下午，被告人邱海在宁江区锦东小区门前以人民币200元的价格将0.4克冰毒卖给白某供其吸食。2016年7月某日下午，被告人邱海在宁江区尚品阳光小区门前以人民币300元的价格将0.4克冰毒卖给白某供其吸食。2016年7月某日晚，被告人邱海在欧亚商场门前以人民币200元的价格将0.4克冰毒卖给白某供其吸食。2016年8月初某日，被告人邱海在松花江大桥江北桥头附近以人民币200元的价格将0.4克冰毒卖给白某供其吸食。2016年8月10日23时许，被告人邱海在松原市经济技术开发区乾元宾馆608房间内，以人民币200元的价格将0.4克冰毒卖给白某供其吸食。针对上述指控，公诉机关向本院提交了书证；证人证言；被告人邱海的供述与辩解等证据。并认为被告人邱海明知是毒品而进行贩卖，作案5起，情节严重，其行为已触犯《中华人民共和国刑法》第三百四十七条第四款之规定，应当以贩卖毒品罪追究其刑事责任。鉴于被告人邱海刑满释放后五年内重新犯罪，属累犯，应依据《中华人民共和国刑法》第六十五条之规定，应当从重处罚；鉴于被告人邱海揭发他人犯罪，查证属实，属立功，应依据《中华人民共和国刑法》第六十八条之规定，可以从轻或者减轻处罚。被告人邱海对公诉机关指控第五起犯罪事实及罪名无异议，对前四起指控均否认。经审理查明，2016年7月某日下午，被告人邱海在宁江区锦东小区门前以人民币200元的价格将0.4克冰毒卖给白某供其吸食。2016年7月某日下午，被告人邱海在宁江区尚品阳光小区门前以人民币300元的价格将0.4克冰毒卖给白某供其吸食。2016年7月某日晚，被告人邱海在欧亚商场门前以人民币200元的价格将0.4克冰毒卖给白某供其吸食。2016年8月10日23时许，被告人邱海在松原市经济技术开发区乾元宾馆608房间内，以人民币200元的价格将0.4克冰毒卖给白某供其吸食。上述事实，有经庭审举证、质证的下列证据予以证实：书证1、抓获经过载明，2016年8月11日16时30分许，松原市公安局宁江二分局石化派出所工作人员在松原市龙嘉宾馆629房间将邱海抓获。2、常住人口信息及现实表现载明，被告人邱海的自然身份情况，在辖区居住期间未发现前科劣迹行为。3、扣押清单、被告人邱海使用农业银行卡照片及微信转款截图载明，抓获邱海后扣押其使用农业银行卡（卡号6228483738186772879）,2016年8月10日晚史某往该卡中转款200元给邱海用于买毒品。4、行政处罚决定书载明，邱海因吸食毒品，于2012年3月1日和2014年2月23日分别被松原市公安局宁江一分局行政拘留十五日和二十日。5、强制隔离戒毒决定书载明，邱海因吸食毒品，于2014年3月12日被松原市安局宁江一分局强制隔离戒毒二年。6、受案登记表、立案决定书、拘留证、批准逮捕决定书、逮捕证载明，2016年8月11日，宁江二分局石化派出所抓获被告人邱海后，其举报在李某处购买过毒品，公安机关对李想贩卖毒品立案侦查，后对李想采取刑事拘留、逮捕措施。7、刑事判决书、释放证明载明，邱海因犯容留他人吸毒罪，于2011年4月18日被松原市宁江区人民法院判处有期徒刑六个月，并处罚金5000元；李某因贩卖毒品罪，于2016年12月15日被松原市宁江区人民法院判处有期徒刑一年，并处罚金5000元8、办案说明载明，刘某1及妻子未找到；王某未找到；白某未找到；辛某未找到。9、办案说明载明，无法找到白某、史某汇款给邱海的银行卡号；1479825****手机号给白某（1360448****）打电话：2016年7月1日13时06分38秒、42分39秒各一次；2016年8月3日18时45分01秒、50分59秒各一次。二、证人证言1、证人白某的证言证实，我和邱海认识十多年了，和刘某认识一年多。我和刘某在邱海手买毒品五次。第一次大约是一个多月之前的一天下午，我打电话给刘某说买冰毒去我朋友家吸，我让刘某去锦东小区门口等我。我就给邱海（1479825****）打电话买200元的冰毒，我用我的微信号给邱海转200元钱，并让邱海把冰毒送到锦东小区正门。我打车到锦东小区刘某已经到了，我下车看见邱海也到了，他给我用一元纸币包着的冰毒就走了。我领着刘某到锦东小区朋友刘某1家，与刘某1夫妻我们四个人把0.4克左右冰毒吸食了。过一两天，我联系邱海买冰毒，邱海让我到尚品阳光小区门口等他。我和刘某去后约20分钟海去了，我给邱海300元现金，邱海给我一个一厘米见方的无色透明塑料包装袋，之后我和刘某打车到江北山下团结南区辛某家，当时就他一个人在家，我拿出冰毒大约0.4-0.5克，我看量有点儿少非常不高兴，就给邱海打电话，邱海说一会再给我送点，我和刘某、辛某就把冰毒吸了。又隔了四五天的晚上十点左右，我朋友武某开红色中兴田野吉普车拉着我从江北去江南刚下一桥在引桥上车没油了，我给刘某打电话让买一矿泉水瓶汽油，刘某来说加油站不卖小瓶汽油。这时我想吸毒，就给邱海打电话联系买200元冰毒，并告诉邱海我在一桥南头路边等他。邱海来时我和刘某、武某在车旁边站着，因为没有现金，我就用刘某手机登录我的微信号（手机号1360448****）给邱海转200元，邱海给我一个一厘米见方的无色透明塑料包装袋冰毒，能有0.4克左右，刘某就走了，我和武某在内把冰毒吸了。七八天前的一天半夜，我联系邱海买200元的冰毒，因我没钱就联系刘某来江北，邱海用短信让我把钱汇到他工行卡里（卡号没记住，短信删除），我把邱海提供的卡号发给刘某，刘某汇完钱就到江北九号公馆附近找我，我俩等邱海挺长时间也没来，刘某走了，我也回家了，第三天下午一点多我和邱海在一桥江北桥头见面才给我一个一厘米见方的无色透明塑料包装袋冰毒，能有0.4克左右，我自己回家吸了。2016年8月10日晚上8点多，我给刘某打电话（1594388****）想买冰毒。刘某来后我俩溜达一会，期间我给邱海打电话（1335137****）及短信联系买冰毒。晚上10点左右，邱海让我到开发区乾元大酒店608室找他。我就和刘某去了，邱海让我们等着，我和刘某去了江北东镇B区姜峰家。大约10点30分左右，邱海说有冰毒了，我说买200元的，我就让刘某给邱海提供的银卡（卡号6228483738186772879）转款200元，又等了一会儿邱海也没打电话，刘某有事走了，我就在姜峰家呆着。8月11日凌晨2点左右，我联系刘某又去了乾元大酒店608室找到邱海，邱海说不好意思让你们等这么长时间，然后把卖给我的冰毒（约0.4克）拿出来我们吸食了大部分，之后我和刘某就回家了。早晨7点多，邱海把吸剩下的冰毒（约0.1克）给我送来，我自己在家又吸食了。2、证人史某的证言证实，我还有一个别名叫刘某。我认识小哥有半年了，通过小哥认识邱海二个月左右，小哥真名及家是哪的我不知道，电话是1360448****。我每次见邱海都是小哥向他买冰毒，见了5、6次吧。二个月之前的一天下午四、五点左右，我和小哥要买冰毒吸，小哥让我到锦东小区门口等他。我去后等四五分钟小哥到了，邱海也来了，我、小哥、邱海走到到锦东小区一栋楼下时邱海走了，我和小哥上了这栋楼的顶楼（应该是六楼）右门那家，男女主人和一个小男孩。小哥从兜里拿出一元纸币包着的冰毒（大约1克左右），我和小哥及男女主人把冰毒吸了，吸食过程中小哥和男主人聊天时我听出冰毒是从邱海那买的。第一次后的一天中午，我和小哥到江南一个小区门口，大约20分钟邱海去了，他们说了两句话邱海就走了,我和小哥去了江北山下小哥的朋友家（一个小区四楼右门），小哥拿出一个一厘米见方的无色透明塑料里面大约有0.7-0.8克冰毒，小哥说给的东西少就给邱海打电话，邱海说一会儿再给送点，我和小哥及小哥的朋友把冰毒吸了。这次后的五、六天的晚上七、八点钟，小哥给我打电话说他和一个朋友开车到江南下一桥快到欧亚那车没油了，让我给送点汽油。我去加油站不卖给我，我就到小哥那和他聊天。不一会儿邱海来，小哥用我的手机上了自己的微信号给邱海转200元买冰毒，邱海给小哥一个一厘米见方的无色透明塑料包装袋冰毒，能有0.2-0.3克，之后我就回家了。一个星期前的一天半夜，小哥给我打电话让我到江北找他，还让我给邱海转点钱买冰毒吸，我打车往江北走到一桥上小哥给我发一个银行卡号，我在江北一下桥的一个工行给邱海银行卡汇了200元，然后我到江北九号公馆门前见到小哥，邱海挺长时间也没来送冰毒，我等不及走了，第二天小哥和我说邱海将货给他了。昨天晚上8、9点钟，小哥找我出来找邱海买冰毒，我俩到乾元宾馆见到邱海，邱海当时没有冰毒，我就回家了。大约晚上11点左右，小哥打电话让我给邱海转200元钱买冰毒，小哥用微信将邱海一个农行卡号发给我，我用支付宝给邱海的银行卡转款200元。今天早上，小哥微信上说邱海给他冰毒了，想给我点儿，后我没接着电话，他就回江北了。今天13时左右，我和邱海微信聊天，他让我到龙嘉公寓629房间，我去后准备吸毒就被警察抓了。3、证人武某的证言证实，白某是我朋友，认识20多年了。我开的红色中兴田野吉普车（吉JA14**）是在长岭农村包自来水工程丛某的。大约一个月前的一天晚上，我开车拉着白某从江北去江南在一桥引桥上车没油了，白某给一个女的打电话让买点散装汽油，大约一个小时那个女的来也没带汽油来，她跟白某在车下聊了一会儿打车走了，至于还有没有其他人出现我记不清了，我没吸食过毒品。4、证人宗某的证言证实，我还叫宗大力，不认识白某，没有卖毒品给白某。三、被告人邱海的供述与辩解，2016年8月10日晚上10点多，我在开发区乾元宾馆608室呆着，小琪跟我微信天说小哥（白某）要买冰毒。我说我这有，小琪说小哥没钱，她替小哥给我200元，11点多小琪用支付宝往我农行卡里转了200元钱。8月11日凌晨1点多，小琪和小哥一起来到乾元宾馆608室，我把一小袋冰毒（约3分）给小哥了，我和小琪、小哥把冰毒吸了。这袋冰毒是8月10日上午在巴特尔附近大街上王志给我的。我认识小哥三、四年了，就卖给他这一次冰毒。上述证据来源合法，部分证据间形成了证据链条，足以证明本院查明被告人邱海的犯罪事实成立，本院对上述证据予以确认。关于被告人邱海对公诉机关指控的第一起、第二起、第三起、第四起贩卖毒品无罪的自行辩解辩护意见，经查，第一起、第二起、第三起，证人白某、史某均证实在被告人邱海处买毒品，且前两起购买的毒品与他人共吸食了，被告人否认但没有证据支持。该辩解辩护意见，不予采纳。公诉机关指控第四起犯罪只有白某证实，属孤证，不能认定被告人邱海犯罪，该辩解辩护意见，予以支持。本院认为，被告人邱海无视国家对毒品的管理制度和人们的生命健康，明知是毒品而贩卖，作案四起起，情节严重，其行为已构成贩卖毒品罪。公诉机关该指控事实清楚，证据确实、充分，指控的罪名成立，本院予以支持。但公诉机关指控被告人邱海第四起贩卖毒品，因缺乏确实充分的证据，指控欠妥，应予纠正。鉴于被告人邱海刑满释放后五年内重新犯罪，属累犯，依法应当从重处罚；鉴于被告人邱海揭发他人犯罪，查证属实，属立功，依法可以从轻处罚。庭审中被告人邱海部分认罪，可酌情从轻处罚。根据被告人邱海的犯罪事实、性质、情节和对社会的危害程度经本院审判委员会2017年5月16日（2017）第十三次会议讨论决定，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第七款，第六十五条、第六十八条之规定，判决如下：被告人邱海犯贩卖毒品罪，判处有期徒刑四年，并处罚金人民币10000元（刑期自判决执行之日起开始计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，刑期自2016年8月15日始至2020年8月14日止；罚金于判决生效后三个月内缴纳）。如不服本判决，可在接到判决书的第二天起十日内，通过本院或直接上诉于吉林省松原市中级人民法院，书面上诉的，应提交上诉状正本一份，副本八份。审　判　长　　王彦军人民陪审员　　张雪芹人民陪审员　　姚立艳二〇一七年五月十六日书　记　员　　王宇祺 更多数据：搜索“马克数据网”来源：www.macrodatas.cn"

text_16 = "云南省福贡县人民法院刑 事 判 决 书(2017)云3323刑初13号公诉机关云南省福贡县人民检察院。被告人罗韦林，曾用名罗维宁，男，1962年3月23日生，彝族，云南省泸水县人，高中文化，无业。2001年7月5日因犯滥用职权罪，被泸水县人民法院判处有期徒刑二年。2016年12月2日因涉嫌犯非法持有毒品罪被福贡县公安局刑事拘留，同年12月16日经福贡县人民检察院批准被逮捕。现羁押于福贡县看守所。辩护人李红枣，云南凡人律师事务所律师。福贡县人民检察院以福检刑检刑诉〔2017〕15号起诉书，指控被告人罗韦林犯非法持有毒品罪，于2017年3月2日向本院提起公诉，本院受理后，依法组成合议庭，于2017年5月2日在本院审判法庭（1）公开开庭审理了本案。福贡县人民检察院指派检察员杨晓萍、助理检察员胡晔骅出庭支持公诉，被告人罗韦林及其辩护人李红枣到庭参加诉讼。现已审理终结。福贡县人民检察院指控：2016年12月1日17时20分许，被告人罗韦林在泸水县六库镇新城区“怒江明珠”北侧附近，从一名叫“腊某某”（在逃）的男子手中以人民币二万五千元的价格购买了两坨毒品可疑物。当被告人罗韦林准备驾车离开时被侦查人员发现，遂即驾车逃离。被告人罗韦林驾车到“怒江明珠”南侧门口便道尽头时下车逃跑，逃跑过程中被告人罗韦林将手中的毒品可疑物丢弃，遂即侦查人员将罗韦林抓获，当场从罗韦林丢弃的棕黄色手提袋中查获毒品可疑物两坨。经鉴定，查获的毒品可疑物为鸦片。经称量净重为2517克。公诉机关针对指控向法庭提交了书证、物证、证人证言、称量记录、鉴定意见、被告人的供述与辩解等证据，认为被告人罗韦林非法持有鸦片2517克的行为触犯了《中华人民共和国刑法》第三百四十八条之规定，构成非法持有毒品罪，应当依法追究其刑事责任。建议判处被告人罗韦林有期徒刑七年零六个月至九年，并处罚金。被告人罗韦林对公诉机关指控的犯罪事实无异议，自愿认罪，希望法庭考虑其持有的鸦片是准备用于给其母亲治病，其母亲年迈且患肺癌晚期需要照顾等特殊情况，对其从轻处罚。辩护人对公诉机关指控被告人罗韦林犯非法持有毒品罪无异议，提出如下量刑意见：1.被告人罗韦林具有依法应当从轻处罚的坦白情节。本案中，被告人罗韦林自被采取强制措施以来，积极配合侦查机关破案，提供线索。在侦查阶段、审查起诉阶段，包括今天的庭审中都如实供述自己的犯罪行为，而且交待毒品卖家腊某某，属于坦白。根据《量刑指导意见》的规定，坦白可以减少基准刑的20%以下。2.被告人罗韦林持有的毒品数量虽然较大，但是尚未流入社会，对社会的危害不大。毒品数量只是定罪量刑的重要情节，而不是唯一情节，量刑还应当综合考虑犯罪情节、危害后果、主观恶性以及人身危险性等各种因素。本案中，被告人罗韦林持有的毒品已被当场查获并收缴，未流入社会，其购买毒品的目的是给其患病的母亲治病，并非牟取暴利，也不是自己吸食，因此，其社会危害性相对较小。3.被告人罗韦林当庭认罪。根据《最高人民法院、最高人民检察院、司法部关于适用普通程序审理被告人认罪案件的若干意见》第九条规定，人民法院对自愿认罪的被告人，酌情予以从轻处罚。4.本案存在公安机关诱惑侦查情形。（1）腊某某刚把毒品放在被告人罗韦林的车上，警察就已经站在被告人驾驶的车旁，并将罗韦林抓获。显然侦查人员已经对被告人罗韦林向腊某某购买鸦片的时间地点已掌握，但侦查人员却不抓卖家腊某某，直接走到被告人罗韦林车旁，因此，侦查人员有选择性执法的嫌疑。（2）根据《中华人民共和国刑事诉讼法》第一百五十一条的规定，经公安机关负责人决定，可以由有关人员隐匿其身份实施侦查，但是，不得诱使他人犯罪。又根据《全国部分法院审理毒品犯罪案件工作座谈会纪要》第六条规定，对特情介入侦破的毒品案件，要区别不同情形分别处理。行为人本没有实施毒品犯罪的主观故意，而是在特情诱惑和促成下形成犯意，进而实施犯罪的，属于“犯意引诱”。对因“犯意引诱”实施毒品犯罪的被告人，根据罪刑相适应原则，应当依法从轻处罚。行为人本来只有实施数量较小的毒品犯罪故意，在特情引诱下实施了数量较大毒品犯罪的，属于“数量引诱”。对因“数量引诱”实施毒品犯罪的被告人，应当依法从轻处罚。（3）本案中，被告人罗韦林购买鸦片欲用于给其母亲治病，因此其主观上仅想购买半斤或者更少量鸦片，但卖家提出要买就买十斤左右，不然卖家没有赚头，最后，经双方多次协商，被告人罗韦林不得已向腊某某购买了五斤。这属于上述第（2）项中的“数量引诱”。被告人罗韦林仅想购买少量鸦片，若数量达不到立案标准，则被告人罗韦林没有犯罪的故意，但在特情的引诱下购买了数量较大的鸦片从而入罪，属于“犯意引诱”。5.综上，由于本案存在公安机关诱惑侦查情形，根据本案的社会危害程度和被告人的悔罪表现，请法庭对被告人罗韦林在量刑时依法从轻、减轻处罚。经审理查明：2016年12月1日17时40分左右，被告人罗韦林按照与卖家腊某某的约定，驾驶车牌号为云xxxx**大众polo轿车，到六库镇新城区四十米大道“怒江明珠”一期北侧门口附近的公路边与腊某某见面。卖家腊某某从一辆农村客运面包车上下来后，坐到被告人驾驶的轿车副驾驶位上，被告人罗韦林交付剩余的五千元后，腊某某回到面包车上将放有涉案鸦片的棕黄色袋子拿出，再次走到被告人罗韦林驾驶的轿车旁打开后排座门，将涉案毒品丢到后排座位上后离开。被告人罗韦林准备驾车离开时，一名侦查人员由南向北迎面走来，并表示要搭乘被告人的车到新城区，被告人罗韦林因害怕被发现其持有毒品，驾车逃逸，被赶来的警车追赶。被告人罗韦林驶到“怒江明珠”二期一便道口时，沿便道逃逸，因车驶到便道尽头没了路，被告人罗韦林下车逃跑，并将涉案毒品丢弃到便道旁的草丛中，随即被抓获。经被告人罗韦林指认，从被告人罗韦林逃跑过程中丢弃的棕黄色袋子内查获两坨鸦片。经称量涉案鸦片净重2517克。2016年11月26日14时32分，被告人罗韦林往卖家腊某某的卡号为XXXXXXXXXXXXXXX的农行账户内存入购买鸦片款二万元人民币。另，被告人罗韦林系非吸毒人员。上述事实有公诉机关向法庭提交的，并经庭审举证、质证的如下证据予以证明：1.抓获经过及同步录音录像。证明怒江州公安局禁毒支队侦查人员根据前期获取的情报线索，在六库镇新城区四十米大道口至新城区路段开展公开查缉。侦查人员先将一名侦查员留在怒江明珠一期前的公路边作为观察哨后，继续沿四十米大道绕行。2016年12月1日17时40分许，侦查人员驾驶的车辆回到四十米大道口转盘东边时，看到前方60米处一辆黑色轿车突然启动并加速往南行驶，遂侦查人员驾车追赶,在怒江明珠南侧（怒江明珠二期）便道尽头追上逃逸的车辆，将被告人罗韦林抓获，并从被告人罗韦林在逃跑中丢弃的一棕黄色袋子里查获两坨毒品可疑物。2.被告人罗韦林的户籍证明。证明被告人罗韦林的基本身份信息。3.检查笔录及照片、毒品称量及提取笔录及照片。证明查获的用于装毒品的手提袋、毒品包裹物、两坨毒品可疑物的外观特征，以及涉案毒品净重为2517克。侦查人员从该两坨毒品中提取了重量分别为7.98克及3.88克的两份检材以备送检的事实。4.怒江州质量技术监督综合检测中心出具的检定证书。证明侦查人员用于称量毒品可疑物的电子天平合格的事实。5.云南省怒江傈僳族自治州公安司法鉴定中心的“（怒）公（司）鉴（理）字〔2016〕160号”鉴定意见。证明上述证据3中送检的两份检材中均检出可待因、吗啡、蒂巴因、罂粟碱、那可汀成分，其中吗啡平均含量为5.15%。6.提取物证登记表、扣押决定书、扣押笔录及照片、扣押清单。证明侦查人员抓获被告人罗韦林后，从被告人罗韦林身上扣押了毒品包裹物共6层，鸦片2517克，金立牌玫瑰金色直板手机一部，车牌号为云XXXX**的黑色大众polo轿车;存款账号为XXXXXXXXXX，存款户名为腊某某，存款金额为二万元人民币，存款时间为2016年11月26日14时32分的农行存款回单一份。7.证人蒙某某的证言。证明涉案黑色大众polo轿车系其所有。8.机动车登记证书、机动车行驶证、发还清单。证明案发时涉案云XXXX**大众polo黑色轿车系蒙某某所有，且已于2017年1月12日发还给蒙某某的事实。9.现场勘验检查笔录、现场勘验方位示意图、现场照片。证明抓获被告人罗韦林的现场位于泸水县六库镇新城区怒江明珠二期南侧便道尽头。10.辨认笔录及照片。证明经被告人罗韦林辨认向其贩卖毒品的男子是片马镇岗房村委会岗房村二组人腊某某。11.证人罗某的证言。证明被告人罗韦林系其父亲，其父亲罗韦林在片马做矿石生意，其祖母祝某某（被告人罗韦林的母亲）患有肺癌晚期。其不清楚其父罗韦林购买鸦片的用途，但其曾听长辈们说鸦片可以止疼。12.现场检测报告书。证明被告人罗韦林系非吸毒人员。13.随案移送物品清单。证明随案移送作案工具金立牌手机一部。14.云南省泸水县人民法院的“（2001）泸刑初字第28号”刑事判决书。证明被告人罗韦林因犯滥用职权罪，于2001年7月5日被泸水县人民法院判处有期徒刑二年。15.祝某某的病历本、诊断证明、出院证明。证明祝某某患有肺癌的事实。被告人罗韦林对上述证据无异议，且未向法庭提交证据。辩护人对抓获经过提出，侦查人员存在选择性执法的嫌疑，因为当时腊某某乘坐的面包车便停放在罗韦林驾驶的轿车前方，侦查人员越过腊某某乘坐的车直接走到罗韦林驾驶的轿车旁，因此，是故意不抓卖家腊某某的。对被告人的供述与辩解提出，侦查人员故意不记录被告人罗韦林讲述的“其只想购买少数鸦片，因腊某某提出要买五斤以上才卖。”以及“腊某某于当日16时40分许打电话给被告人时说腊某某在“双米底”（地名），但腊某某17时20分许便再次打电话告知被告人罗韦林其已到六库，在时间上明显不合理。”的内容。辩护人对其余证据未提出异议意见。辩护人向法庭提交了被告人罗韦林母亲祝某某的病历本及诊断证明。证明祝某某于2013年3月份确诊为肺癌晚期的事实。公诉人对辩护人提交的证据无异议。辩护人针对“抓获经过”、“被告人的供述与辩解”分别提出的，侦查人员“选择性执法”、“选择性记录”的质证意见无事实依据，不予采纳。上述证据经庭审举证、质证，形式合法、内容客观、证据间相互印证，足以证明本案经审理查明的事实，本院予以确认。本院认为，被告人罗韦林明知是鸦片仍向他人购买并持有的行为符合《中华人民共和国刑法》第三百四十八条之规定，构成非法持有毒品罪。公诉机关指控的罪名成立。被告人罗韦林持有的鸦片净重2517克，法定刑为七年以上有期徒刑或者无期徒刑，并处罚金。被告人罗韦林到案后至庭审，均能如实供述自己的罪行，认罪悔罪，依法予以从轻处罚；被告人罗韦林始终供述称其购买鸦片用于治疗、缓解其母亲祝某某肺癌病痛，祝某某确实患有肺癌，因此，本案中不能排除被告人罗韦林购买鸦片用于给其母亲治病的可能性，与其他购买后吸食的非法持有毒品犯罪相比，社会危害程度小，酌情予以从轻处罚。辩护人关于公安机关对被告人罗韦林“诱惑侦查”即“犯意引诱”的辩护意见，因被告人罗韦林多次主动打电话给腊某某，表示欲向腊某某购买鸦片，并对购买价格、数量进行商议，因此该辩护意见本院不予采纳。辩护人的其余辩护意见，本院量刑时予以考虑。综合本案的犯罪事实、情节及社会危害程度，根据罪刑法定、罪刑相适应原则，依照《中华人民共和国刑法》第三百四十八条、第六十七条第三款之规定，判决如下：一、被告人罗韦林犯非法持有毒品罪，判处有期徒刑七年零六个月，并处罚金七千元；（刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2016年12月2日起至2024年6月1日止，罚金限判决生效后一个月内一次缴清。）二、随案移送的作案工具金立手机一部，依法没收存档。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向云南省怒江傈僳族自治州中级人民法院提出上诉。书面上诉的，提交上诉状正本一份，副本二份。审　判　长　彭佩珍&#xB;审　判　员　李荣华&#xB;人民陪审员　肖　　 胡　　 花二〇一七年五月四日书　记　员　桑　　 丽　　 丽 马 克 数 据 网"

text_17 = "广东省清远市清城区人民法院刑 事 判 决 书（2017）粤1802刑初156号公诉机关清远市清城区人民检察院。被告人徐健洪，男，1989年11月11日出生，汉族，文化程度初中，无业，户籍所在地：广东省清远市清城区。因本案于2016年7月14日被刑事拘留，同年8月19日被逮捕。现羁押于清远市清新区看守所。辩护人毛福清、郑婷婷，分别是广东汇棋律师事务所律师、实习律师。被告人谭某林，男，1984年9月2日出生，汉族，文化程度初中，无业，户籍所在地：广东省清远市清城区。因本案于2016年7月14日被刑事拘留，同年8月19日被逮捕。现羁押于清远市清新区看守所。辩护人朱绍敏，广东法行律师事务所律师。被告人彭某瑜，男，1989年11月17日出生，汉族，文化程度初中，无业，户籍所在地：广东省清远市清城区，住广东省清远市清城区。因犯故意伤害罪于2009年9月22日被清远市清城区人民法院判处有期徒刑一年，缓刑一年，缓刑考验期至2010年9月29日止。缓刑考验期间因违反法倖规定，于2010年4月15日被清远市清城区人民法院裁定收监执行原判刑罚，同年12月3日刑满释放。因犯贩卖毒品罪于2013年8月6日被清远市清城区人民法院判处有期徒刑六个月，同年9月22日刑满释放。因本案于2016年7月14日被刑事拘留，同年8月19日被逮捕。现羁押于清远市清新区看守所。清远市清城区人民检察院以清城检诉刑诉（2017）150号起诉书指控被告人徐健洪、谭某林、彭某瑜犯贩卖毒品罪，于2017年3月16日向本院提起公诉。本院受理后，依法组成合议庭，于2017年4月26日公开开庭进行了审理，清远市清城区人民检察院指派检察员黄洁明出庭支持公诉，被告人徐健洪、谭某林、彭某瑜，辩护人毛福清、郑婷婷、朱绍敏到庭参加了诉讼，现已审理终结。经审理查明：一、贩卖毒品罪2016年7月初，被告人徐健洪在清远市清城区东城街道东城御锋后门路段，清远市清城区大学西路莲塘新市场附近，先后两次共将0.8克的毒品甲基苯丙胺贩卖给被告人彭某瑜，共收取人民币200元。2016年7月初的一天，被告人徐健洪在清远市清城区桥北路回元堂门口附近，将0.2克的毒品甲基苯丙胺贩卖给吸毒人员林某1，收取人民币100元。同年7月14日，公安民警在清远市清城区东城街道东城御锋后门附近将被告人徐健洪抓获，并当场在其驾驶的小车内起获毒品甲基苯丙胺21.78克。2016年7月初的一天，被告人彭某瑜在清远市清城区东城街道莲发15巷巷尾，将1克的毒品氯胺酮贩卖给吸毒人员朱某1，收取人民币200元。2016年7月初的一天，被告人彭某瑜在清远市清城区东城街道莲发22巷41号楼下超市旁，将0.5克的毒品氯胺酮贩卖给被告人谭某林，收取人民币100元。同年7月14日，被告人彭某瑜协助公安民警抓获了其他被告人。2016年7月初，被告人谭某林在清远市清城区东城街道莲发22巷乐嘉嘉超市附近、清远市清城区桥北路苏荷酒吧对面路边，先后两次共将2.5克氯胺酮贩卖给吸毒人员朱某1，共收取人民币400元。同年7月13日，公安民警在清远市清城区东城街道莲发22巷乐嘉嘉超市附近将被告人谭某林抓获，并当场在其车上起获毒品氯胺酮0.16克。二、掩饰、隐瞒犯罪所得罪2016年6月底某天晚上22时许，被告人谭某林在清远市清城区东城街道名德里小区附近的某道路旁，以人民币8500元明显低于市场的价格向林志威（另案处理）购买一辆没有合法有效来历凭证的江铃牌工具车（价值人民币47767元）。案发后，公安人员在谭某林处起获涉案车辆，并已发还给被害人邓某1。上述事实，被告人徐健洪、谭某林、彭某瑜在法庭审理过程中无异议，并有物证：可疑白色晶体状物1包等；书证：户籍材料、通话记录、现场检测报告书；证人林某1、朱某1的证言；被告人的供述与辩解：被告人徐健洪、彭某瑜、谭某林的供述与辩解；鉴定意见；检查笔录、辨认笔录、现场勘查笔录；视听资料、电子数据等证据证实，足以认定。被告人徐健洪的辩护人辩称，被告人徐健洪被动作案，具有犯意引诱，依法应当从轻处罚。被告人徐健洪是初犯，归案后如实供述自己的犯罪事实，自愿认罪，依法应予以从轻处罚。被告人谭某林的辩护人辩称，被告人谭某林贩卖毒品数量很少，主观恶性轻，没有获得任何利益。被告人谭某林收购的车已经退赃，可以从轻处罚。被告人谭某林是初犯，到案后认罪态度良好。本院认为，被告人徐健洪无视国家法律，贩卖毒品，数量较大，其行为触犯了《中华人民共和国刑法》第三百四十七条第三款之规定，构成贩卖毒品罪。被告人谭某林无视国家法律，贩卖少量毒品，明知是犯罪所得而予以收购，其行为分别触犯了《中华人民共和国刑法》第三百四十七条第四款、第三百一十二条之规定，构成贩卖毒品罪、掩饰、隐瞒犯罪所得罪，依法应当数罪并罚。被告人彭某瑜无视国家法律，贩卖少量毒品，其行为触犯了《中华人民共和国刑法》第三百四十七条第四款之规定，构成贩卖毒品罪。公诉机关指控被告人徐健洪犯贩卖毒品罪，被告人谭某林犯贩卖毒品罪、掩饰、隐瞒犯罪所得罪，被告人彭某瑜犯贩卖毒品罪，犯罪事实清楚，证据确实、充分，本院应予支持。被告人彭某瑜曾因故意犯罪被判处有期徒刑，在刑罚执行完毕后，五年内再犯应当判处徒刑以上刑罚之罪，是累犯，应当从重处罚。被告人彭某瑜因贩卖毒品罪被判过刑，又再犯贩卖毒品罪，是毒品再犯，从重处罚。被告人彭某瑜归案后协助司法机关抓获其他被告人，是立功，可以予以从轻处罚。被告人徐健洪、谭某林、彭某瑜归案后能够基本如实供述自己的犯罪事实，依法予以从轻处罚。对于被告人徐健洪的辩护人提出，被告人徐健洪具有犯意引诱的辩护意见，经查属实，可对被告人徐健洪从轻处罚。对于被告人徐健洪的辩护人提出，被告人徐健洪是初犯，归案后如实供述自己的犯罪事实，自愿认罪的辩护意见，经查属实，本院予以采纳。对于被告人谭某林的辩护人提出，被告人谭某林贩卖毒品数量很少，主观恶性轻，没有获得任何利益的辩护意见，根据法律规定，贩卖毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚，因此，对辩护人这一辩护意见，本院不予采纳。对于被告人谭某林的辩护人提出，被告人谭某林收购的车已经退赃，可以从轻处罚以及被告人谭某林是初犯，到案后认罪态度良好的辩护意见，经查属实，本院予以采纳。综上所述，根据各被告人的犯罪事实、情节以及悔罪表现，依照《中华人民共和国刑法》第三百四十七条第三款、第四款、第三百一十二条、第三百五十六条、第六十四条、第六十五条、第六十七条第三款、第六十八条、第六十九条之规定，判决如下：一、被告人徐健洪犯贩卖毒品罪，判处有期徒刑七年六个月，并处罚金12000元（罚金在判决生效一个月内交纳）。（刑期自判决执行之日起计算，判决执行以前先行羁押的羁押一日折抵刑期一日，即徐健洪的刑期自2016年7月14日起，执行至2024年1月13日止。）二、被告人谭某林犯贩卖毒品罪，判处有期徒刑九个月，并处罚金2000元，犯掩饰、隐瞒犯罪所得罪，判处有期徒刑十个月，并处罚金3000元，合并总和刑期一年七个月，决定执行有期徒刑一年六个月，并处罚金5000元（罚金在判决生效一个月内交纳）。（刑期自判决执行之日起计算，判决执行以前先行羁押的羁押一日折抵刑期一日，即谭某林的刑期自2016年7月14日起，执行至2018年1月13日止。）三、被告人彭某瑜犯贩卖毒品罪，判处有期徒刑一年，并处罚金3000元（罚金在判决生效一个月内交纳）。（刑期自判决执行之日起计算，判决执行以前先行羁押的羁押一日折抵刑期一日，即彭某瑜的刑期自2016年7月14日起，执行至2017年7月13日止。）四、随案移送的粤Ｒ×××××牌号东南菱帅小车1辆（暂存清远市公安局清城区分局），是作案工具，依法没收，上缴国库；手机1台（已坏）、车牌2块，是作案工具，依法没收销毁。如不服本判决，可在接到判决书的第二日起十日内，通过本院或直接向广东省清远市中级人民法院提出上诉。书面上诉的，应交上诉状正本一份，副本三份。审　判　长　　肖国献人民陪审员　　黄桂娟人民陪审员　　黎子权二〇一七年五月五日书　记　员　　李振鹏相关法律条文：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第三百一十二条明知是犯罪所得及其产生的收益而予以窝藏、转移、收购、代为销售或者以其他方式掩饰、隐瞒的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。第三百五十六条因走私、贩卖、运输、制造、非法持有毒品罪被判过刑，又犯本节规定之罪的，从重处罚。第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还；违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。第六十五条被判处有期徒刑以上刑罚的犯罪分子，刑罚执行完毕或者赦免以后，在五年以内再犯应当判处有期徒刑以上刑罚之罪的，是累犯，应当从重处罚，但是过失犯罪和不满十八周岁的人犯罪的除外。前款规定的期限，对于被假释的犯罪分子，从假释期满之日起计算。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。第六十八条犯罪分子有揭发他人犯罪行为，查证属实的，或者提供重要线索，从而得以侦破其他案件等立功表现的，可以从轻或者减轻处罚；有重大立功表现的，可以减轻或者免除处罚。犯罪后自首又有重大立功表现的，应当减轻或者免除处罚。第六十九条判决宣告以前一人犯数罪的，除判处死刑和无期徒刑的以外，应当在总和刑期以下、数刑中最高刑期以上，酌情决定执行的刑期，但是管制最高不能超过三年，拘役最高不能超过一年，有期徒刑总和刑期不满三十五年的，最高不能超过二十年，总和刑期在三十五年以上的，最高不能超过二十五年。 来源：百度搜索“马克数据网”"

text_18 = "安徽省芜湖市镜湖区人民法院刑 事 判 决 书（2017）皖0202刑初105号公诉机关安徽省芜湖市镜湖区人民检察院。被告人金武，男，1983年10月5日出生，户籍地安徽省芜湖市镜湖区。2002年因犯寻衅滋事罪、引诱卖淫罪分别被判处有期徒刑一年六个月、有期徒刑二年，宣告缓刑三年，后撤销缓刑，决定执行有期徒刑三年；2009年因犯贩卖毒品罪被本院判处有期徒刑一年六个月，并处罚金人民币五千元；2015年7月9日因吸毒被芜湖市公安局开发区分局行政拘留十日，并被责令接受社区戒毒三年。2017年1月8日因本案被芜湖市公安局镜湖分局刑事拘留，同年1月16日经芜湖市镜湖区人民检察院批准逮捕，次日由芜湖市公安局镜湖分局执行逮捕。现羁押于芜湖市第一看守所。指定辩护人李长志，安徽春蓝律师事务所律师。指定辩护人张文英，安徽春蓝律师事务所律师。被告人张虹，女，1977年1月31日出生，户籍地安徽省芜湖市镜湖区。2017年1月8日因本案被芜湖市公安局镜湖分局取保候审，同年3月28日被本院继续取保候审。辩护人蒋克江，安徽维晨律师事务所律师。安徽省芜湖市镜湖区人民检察院以镜检刑诉（2017）103号起诉书指控被告人金武、张虹犯贩卖毒品罪，于2017年3月28日向本院提起公诉。本院依法组成合议庭，于2017年5月10日公开开庭审理了本案。芜湖市镜湖区人民检察院指派检察员卫艳出庭支持公诉。被告人金武及其辩护人李长志、张文英、被告人张虹及其辩护人蒋克江到庭参加诉讼。现已审理终结。经审理查明：一、被告人金武贩卖毒品的事实2016年10月，被告人金武先后两次在本市镜湖区王家巷长盛园一棋牌室附近向吸毒人员胡某贩卖毒品甲基苯丙胺（冰毒），每次400元（约0.5克），共计约1克。二、被告人张虹贩卖毒品的事实2016年12月至2017年1月期间，被告人金武三次联系葛某某（在逃）购买毒品甲基苯丙胺，后葛某某将毒品交给其女朋友即被告人张虹，由被告人张虹与金武取得联系后在本市镜湖区华强广场附近，每次以300元的价格贩卖约0.5克的甲基苯丙胺给金武，共计约1.5克，并将收取的毒资交给葛某某。2017年1月7日，被告人金武被公安机关抓获，并于当日协助公安机关抓获被告人张虹。公安机关并从被告人张虹住处查获毒品甲基苯丙胺4.55克。上述事实，被告人金武、张虹在开庭审理中亦无异议，并有证人胡某的证言，书证户籍信息、抓获经过及情况说明、扣押决定书及清单、收缴毒品专用收据、行政处罚决定书、社区戒毒决定书、刑事判决书、病历材料，搜查笔录、称重笔录、辨认笔录，通话详单，刑事照片及鉴定意见等证据证实，足以认定。本院认为，被告人金武违反国家毒品管理规定，向他人贩卖毒品甲基苯丙胺约1克，其行为已构成贩卖毒品罪；被告人张虹伙同他人违反国家毒品管理规定，多次贩卖毒品甲基苯丙胺约6克，情节严重，其行为均已构成贩卖毒品罪，且系共同犯罪。公诉机关指控的罪名成立。被告人金武系毒品再犯且有其他犯罪前科，依法予以从重处罚。被告人金武到案后能如实供述犯罪事实，并协助公安机关抓获其他犯罪嫌疑人，系立功，依法予以从轻处罚。被告人张虹在共同犯罪中起次要作用，系从犯，依法予以减轻处罚。被告人张虹在归案后能如实供述犯罪事实，依法予以从轻处罚。公安机关从被告人张虹的住处查获的毒品甲基苯丙胺4.55克尚未实际流入社会，该情节在量刑时予以考虑。二被告人辩护人相关的辩护意见予以采纳。为维护国家对毒品的管理制度和公民的健康权利，维护社会管理秩序，打击犯罪，根据二被告人的犯罪事实、性质、情节和对社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一、四、七款、第三百五十六条、第二十五条第一款、第二十七条、第六十七条第三款、第六十八条、第五十二条、第五十三条第一款以及《最高人民法院〈关于审理毒品案件定罪量刑标准有关问题的解释〉》第四条第（一）项之规定，判决如下：一、被告人金武犯贩卖毒品罪，判处有期徒刑一年，并处罚金人民币二千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日，折抵刑期一日。即自2017年1月8日起至2018年1月7日止。罚金款于判决生效后十日内缴纳。）二、被告人张虹犯贩卖毒品罪，判处有期徒刑二年，并处罚金人民币三千元。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日，折抵刑期一日。罚金款于判决生效后十日内缴纳。）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向安徽省芜湖市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审　判　长　　孙　玲人民陪审员　　徐孝红人民陪审员　　姚琛嘉二〇一七年五月十日书　记　员　　向玉媛附相关法条：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（一）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第三百五十六条因走私、贩卖、运输、制造、非法持有毒品罪被判刑，又犯本节规定之罪的，从重处罚。第二十五条第一款共同犯罪是指二人以上共同故意犯罪。第二十七条在共同犯罪中起次要或者辅助作用的，是从犯。对于从犯，应当从轻、减轻处罚或者免除处罚。第六十八条犯罪分子有揭发他人犯罪行为，查证属实的，或者提供重要线索，从而得以侦破其他案件等立功表现的，可以从轻或者减轻处罚；有重大立功表现的，可以减轻或者免除处罚。第五十二条判处罚金，应当根据犯罪情节决定罚金数额。第五十三条第一款罚金在判决指定的期限内一次或者分期缴纳。期满不缴纳的，强制缴纳。对于不能全部缴纳罚金的，人民法院在任何时候发现被执行人有可以执行的财产，应当随时追缴。如果由于遭遇不能抗拒的灾祸缴纳确实有困难的，可以酌情减少或者免除。第六十七条第三款犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。最高人民法院《关于审理毒品案件定罪量刑标准有关问题的解释》第四条走私、贩卖、运输、制造毒品，具有下列情形之一的，应当认定为刑法第三百四十七条第四款规定的“情节严重”：向多人贩卖毒品或者走私、贩卖运输、制造毒品的；在戒毒场所、监管场所贩卖毒品的；向在校学生贩卖毒品的；组织、利用残疾人、严重疾病患者、怀孕或者正在哺乳自己婴儿的妇女走私、贩卖、运输、制造毒品的；国家工作人员走私、贩卖、运输、制造毒品的；其他情节严重的情形。 微信公众号“马克 数据网”"

text_19 = "贵州省习水县人民法院刑 事 判 决 书（2017）黔0330刑初141号公诉机关习水县人民检察院。被告人张兴尧，曾用名张正涛，绰号新桃，男，生于1988年12月14日，汉族，初中文化，贵州省习水县人，住习水县，无业。因涉嫌贩卖毒品罪，于2017年2月13日被刑事拘留，于2017年2月21日被释放，后又于2017年2月28日被执行逮捕。现羁押于习水县看守所。习水县人民检察院以习检公诉刑诉（2017）106号起诉书指控被告人张兴尧犯贩卖毒品罪，于2017年5月4日向本院提起公诉。本院受理后，依法组成合议庭公开开庭进行了审理。习水县人民检察院指派检察员袁卫蓉出庭支持公诉，被告人张兴尧到庭参加诉讼。本案现已审理终结。习水县人民检察院指控：被告人张兴尧在“猪儿”处购得甲基苯丙胺嫌疑物。2017年2月3日20时许，吸毒人员陆某通过手机信息联系被告人张兴尧购买1500元的甲基苯丙胺（冰毒）嫌疑物，双方约定在双龙乡双龙村河坝组进行交易。当晚21时许，被告人张兴尧携带甲基苯丙胺嫌疑物到约定地点进行交易时被当场查获。从被告人张兴尧处扣押1.11克甲基苯丙胺嫌疑物，从陆某处扣押2.74克甲基苯丙胺嫌疑物，经遵义市公安司法鉴定中心鉴定，当场查获的甲基苯丙胺嫌疑物中均检出甲基苯丙胺成分。公诉机关认为被告人张兴尧明知是毒品而予以贩卖的行为触犯了《中华人民共和国刑法》第三百四十七条第四款之规定，犯罪事实清楚，证据确实、充分，应当以贩卖毒品罪追究其刑事责任。并建议在有期徒刑六个月至一年六个月幅度内量刑。被告人张兴尧对公诉机关指控的犯罪事实和罪名均无异议，并认罪认罚。上述审理事实，本院予以确认。本院认为：被告人张兴尧为以贩养吸，明知是毒品而贩卖的行为，触犯了《中华人民共和国刑法》第三百四十七条第四款之规定，应当以贩卖毒品罪追究其刑事责任。公诉机关指控被告人张兴尧犯贩卖毒品的犯罪事实及罪名成立，本院予以确认。被告人张兴尧归案后，如实供述了其罪行，属坦白，根据《中华人民共和国刑法》第六十七条第三款，可从轻处罚。被告人张兴尧在执行逮捕前已被刑事拘留9日，执行时应折抵刑期9日。为此，根据被告人张兴尧的犯罪事实、性质、情节、悔罪表现及对社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第四款、第五十二条、第五十三条、第六十七条第三款、第四十七条之规定，判决如下：被告人张兴尧犯贩卖毒品罪，判处有期徒刑一年，并处罚金五千元。（刑期从判决执行之日起计算，判决执行前先行羁押的，羁押一日折抵刑期一日，即自2017年2月28日起至2018年2月18日止；罚金限判决生效后一个月内缴纳。）如不服本判决，可在接到本判决书之次日起十日内，通过本院或者直接向贵州省遵义市中级人民法院提起上诉。书面上诉的，应当提交上诉状正本一份，并按对方当事人的人数提出副本。审　判　长　　黄麒菱人民陪审员　　张　丽人民陪审员　　刘仕德二〇一七年五月二十二日书　记　员　　涂小芳 更多数据：搜索“马克数据网”来源：www.macrodatas.cn"

text_20 = "重庆市南岸区人民法院刑 事 判 决 书（2017）渝0108刑初518号公诉机关重庆市南岸区人民检察院。被告人陈羽，男，1985年2月5日出生于重庆市渝中区，汉族，初中文化，无业，户籍地重庆市渝中区。2011年1月31日因犯贩卖毒品罪被重庆市九龙坡区人民法院判处有期徒刑六个月，同年5月31日刑满��放。2012年3月27日因犯贩卖毒品罪被重庆市渝中区人民法院判处有期徒刑七个月零二日，同年8月6日刑满释放。2013年4月11日因犯贩卖毒品罪被重庆市渝中区人民法院判处有期徒刑一年二个月，2014年3月17日刑满释放。2014年5月30日因犯贩卖毒品罪被本院判处有期徒刑七个月。因本案于2017年3月7日被抓获，次日被刑事拘留，2017年3月18日被执行逮捕。现羁押于重庆市南岸区看守所。重庆市南岸区人民检察院以渝南检公诉刑诉〔2017〕487号起诉书指控被告人陈羽犯贩卖毒品罪，于2017年5月15日向本院提起公诉。本院受理后，依法适用简易程序，实行独任审判，于2017年5月24日公开开庭审理了本案。重庆市南岸区人民检察院指派检察员朱晓宇出庭支持公诉。被告人陈羽到庭参与诉讼。现已审理终结。公诉机关指控：2017年3月7日11时许，购毒人陈某与被告人陈羽��话联系商定了购毒事宜。后陈某通过微信转账的方式支付给陈羽毒资200元。当日14时许，陈羽在重庆市南岸区俊逸第一江岸小区12栋2楼的楼梯间转角处，将净重0.1克甲基苯丙胺片剂（俗称麻古）和0.2克甲基苯丙胺（冰毒）贩卖给陈某。交易完成后，民警将陈羽抓获，并查获了上述涉案毒品。经重庆市公安局毒品检测中心鉴定，从查获的上述毒品中均检测出甲基苯丙胺成分。被告人陈羽归案后，如实向公安机关供述了上述犯罪事实。上述事实，被告人在开庭审理过程中亦无异议，且有户籍资料、捉获经过、扣押决定书及清单、通话清单、微信聊天记录、前科材料等书证，证人陈某的证言，被告人陈羽的供述与辩解，搜查、提取、封存笔录及照片、称量笔录及照片、检材提取笔录及照片，重庆市公安局毒品检测中心物证检验报告等证据予以证实，足以认定。本院认为，被告人陈羽明知是毒品甲基苯丙胺而向他人贩卖共计0.3克，其行为已构成贩卖毒品罪。公诉机关指控的罪名成立，予以支持。陈羽曾因毒品犯罪被判处有期徒刑，在刑罚执行完毕后五年内再犯应当被判处有期徒刑以上之贩卖毒品罪的，系累犯和毒品再犯，依法应从重处罚。归案后，被告人陈羽能如实供述犯罪事实，有坦白情节，依法可从轻处罚。据此，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第五十二条、第五十三条、第三百五十六条、第六十七条第三款、第六十四条之规定，判决如下：一、被告人陈羽犯贩卖毒品罪，判处有期徒刑九个月，并处罚金一千元。（刑期从判决执行之日起计算。判决执行之前先行羁押的，羁押一日折抵刑期一日。即从2017年3月7日起至2017年12月6日止。罚金限本判决生效即缴纳。）二、对扣押在案的毒品依法予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向重庆市第五中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。代理审判员　　吴广哲二〇一七年五月二十四日书　记　员　　林　颖 微信公众号“马克数据网”"

total = 20
texts = []
for x in range(total):
    texts.append(eval(f"text_{x}"))

print(texts)
i = 0

for text in texts:
    print("text" + str(i), "below:")
    i += 1
    print(trim_and_fetch_facts(text)[0])

# test Claude API output, parsing response text.

In [0]:
import os
import pandas as pd

def parse_response_text(response_text):
    """
    Parses the 'ResponseText' to extract the drug and amount information.
    Adds a missing opening bracket, replaces single quotes with double quotes,
    and returns a dictionary with the extracted information.
    """
    # Fix the format of the response_text to make it valid JSON
    corrected_format = "{" + response_text.replace("'", '"')
    try:
        # Parse the corrected string as JSON
        data = json.loads(corrected_format)
    except json.JSONDecodeError:
        # Return None or a dictionary with default values if parsing fails
        return None
    return data


def process_files(base_path, cause):
    global total_rows_count, failed_fetch_count, na_in_response_count
    # Define the output folder based on the base path and cause
    output_folder = f"/mnt/processed_data_criminal_case_analysis/drug_related_Claude_response_March_19"


    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(base_path)

    for file in files:
        # Check if the item is a directory and iterate through its contents
            for sub_file in files:
                if sub_file.name.endswith(".csv"):
                    # Reading CSV file into DataFrame
                    file_path = sub_file.path.replace("dbfs:", "/dbfs")  # Convert to local file path if necessary
                    df = pd.read_csv(file_path, on_bad_lines='skip')

                    
                    # Filter rows where 'TrialProcedure' column contains '一审'
                    df_filtered = df[df['TrialProcedure'].str.contains('一审', na=False)]

                    # Apply 'trim_and_fetch_facts' function to 'FullText' column
                    enriched_data = df_filtered['FullText'].apply(trim_and_fetch_facts)

                    if df_filtered.empty:
                        print(f"No data after filtering for {sub_file.name}. Moving to the next file.")
                        continue

                    if not isinstance(enriched_data.iloc[0], tuple) or len(enriched_data.iloc[0]) != 3:
                        print(f"Data structure mismatch in file: {file_path}")
                        continue
                    
                    # Expanding the enriched_data into separate columns
                    df_expanded = pd.DataFrame(enriched_data.tolist(), columns=['ResponseText', 'TrimmedType', 'TextAroundTrimmedPoint'], index=df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_expanded], axis=1)
                    
                    # Parse the 'ResponseText' and expand it into new columns
                    parsed_data = df_filtered['ResponseText'].apply(parse_response_text)
                    df_parsed = pd.json_normalize(parsed_data)
                    df_filtered = pd.concat([df_filtered, df_parsed], axis=1)
                    
                    # Drop the 'ResponseText' column
                    df_filtered.drop('ResponseText', axis=1, inplace=True)

                    total_rows_count += len(df_filtered)
                    failed_fetch_count += len(df_filtered[df_filtered['ResponseText'] == 'Failed to fetch response from API'])
                    na_in_response_count += len(df_filtered[df_filtered['ResponseText'].str.contains("NA", na=False)])
                    
                    
                    # Construct the output path for the enriched CSV file
                    output_file_path = 'dbfs:'+ os.path.join(output_folder, f"{os.path.basename(sub_file.name)}")
                    
                    # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
                    df_filtered.to_csv(output_file_path.replace("dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis/finetuning_training_data/drug_related_sample_training_data/"
causes_of_action = ["drug_related"]

for cause in causes_of_action:
    total_rows_count, failed_fetch_count, na_in_response_count = 0, 0, 0
    process_files(base_path, cause)

    # Print the counts for process failures.
    print(f"For {cause}, Total rows processed: {total_rows_count}")
    print(f"For {cause}, Failed API fetch count: {failed_fetch_count}")
    print(f"For {cause}, NA in response count: {na_in_response_count}")